In [1]:
import requests
from urllib.request import urlopen
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import numpy.ma as ma
import os, sys
import itertools
from datetime import datetime, timezone
import calendar
import json
from operator import itemgetter
import textwrapper

In [73]:


def parse_timestamp(url):

    '''
    This function will take a url string, and create a datetime object from the timestamp in the url.

    url : str, the API url with the timestamp in the url

    returns:
    timestamp : a datetime object with the url timestamp
    station : str, the three letter station ID

    '''
    
    timestamp = url.split('nwstext/')[-1].split('-KLZK')[0]
    
    y = timestamp[:4]
    m = timestamp[4:6]
    d = timestamp[6:8]
    hh = timestamp[8:10]
    mm = timestamp[10:]

    station = url.split('CLM')[-1]
    
    return datetime(y, m , d, hh, mm), station


    

'53'

In [75]:
url.split('CLM')

'LZK'

In [ ]:
#start small, lets set up a script to go query all the monthly products in a single calendar year...

In [3]:


# # ----------------------------------------------------------
# def parse_timestamp(y, m, d, hh, mm):

#     return datetime(y, m, d, hh, mm)
#     #timestamp = timestamp.astimezone()
#     #return timestamp.astimezone(timezone.utc)

# ----------------------------------------------------------
def count_spaces(line):

    # this should be the date in the line
    split_date = list(filter(None, (line.split(' '))))[0]

    # this is the number of spaces for it to be a date occurred that month
    if line.split(split_date)[0].count(' ') < 30:
        return True

    else:
        return False

# ----------------------------------------------------------

# def get_maxmin_data(high_temp_data, low_temp_data, station, month):

#     '''
#     Worker function to extract high and low data/dates from the CLM products.
    
#     '''

#     month = calendar.month_name[month][:3].upper()
    
#     # first filter the line of temp data
#     filtered_low_temp_data = list(filter(None, low_temp_data.split(' '))) # low temp data
#     filtered_high_temp_data = list(filter(None, high_temp_data.split(' '))) # high temp data
    
#     # lets add the high and low values to the dictionary
#     data_dict[station][month]['lows'].append(filtered_low_temp_data[0])
#     data_dict[station][month]['highs'].append(filtered_high_temp_data[0])

    
#     ## LOW TEMP DATA DATES ##
#     #---------------------------------------------------------------
#     # check how many lines exist in the list
#     # if only one line, then just one date to grab
#     if len(low_temp_data.splitlines()) == 1:
    
#         data_dict[station][month]['low_dates'].append([filtered_low_temp_data[1]])
    
#     # if there are multiple rows of low temp data, then multiple dates to grab
#     elif len(low_temp_data.splitlines()) > 1:
#         dates = []
        
#         # grab the first date in the line
#         dates.append(filtered_low_temp_data[1])
    
#         # now iterate and find the rest of the dates
#         for line in low_temp_data.splitlines()[1:]:
#             if count_spaces(line):
#                 dates.append(list(filter(None, (line.split(' '))))[0])
    
    #     data_dict[station][month]['low_dates'].append(dates)
    
    
    # ## HIGH TEMP DATA DATES ##
    # #---------------------------------------------------------------
    # # check how many lines exist in the list
    # # if only one line, then just one date to grab
    # if len(high_temp_data.splitlines()) == 1:
    
    #     data_dict[station][month]['high_dates'].append([filtered_high_temp_data[1]])
    
    # # if there are multiple rows of high temp data, then multiple dates to grab
    # elif len(high_temp_data.splitlines()) > 1:
    #     dates = []
        
    #     # grab the first date in the line
    #     dates.append(filtered_high_temp_data[1])
    
    #     # now iterate and find the rest of the dates
    #     for line in high_temp_data.splitlines()[1:]:
    #         if count_spaces(line):
    #             dates.append(list(filter(None, (line.split(' '))))[0])
    
    #     data_dict[station][month]['high_dates'].append(dates)

# ----------------------------------------------------------

def text_parser(timestamp, pil):

    def get_maxmin_data(high_temp_data, low_temp_data, station, month):
    
        '''
        Worker function to extract high and low data/dates from the CLM products.
        
        '''
    
        month = calendar.month_name[month][:3].upper()

        print(high_temp_data)
        print(low_temp_data)

        
        # first filter the line of temp data
        filtered_low_temp_data = list(filter(None, low_temp_data.split(' '))) # low temp data
        filtered_high_temp_data = list(filter(None, high_temp_data.split(' '))) # high temp data
        
        # lets add the high and low values to the dictionary
        data_dict[station][month]['lows'].append(filtered_low_temp_data[0])
        data_dict[station][month]['highs'].append(filtered_high_temp_data[0])
    
        
        ## LOW TEMP DATA DATES ##
        #---------------------------------------------------------------
        # check how many lines exist in the list
        # if only one line, then just one date to grab
        if len(low_temp_data.splitlines()) == 1:
        
            data_dict[station][month]['low_dates'].append([filtered_low_temp_data[1]])
        
        # if there are multiple rows of low temp data, then multiple dates to grab
        elif len(low_temp_data.splitlines()) > 1:
            dates = []
            
            # grab the first date in the line
            dates.append(filtered_low_temp_data[1])
        
            # now iterate and find the rest of the dates
            for line in low_temp_data.splitlines()[1:]:
                if count_spaces(line):
                    dates.append(list(filter(None, (line.split(' '))))[0])
        
            data_dict[station][month]['low_dates'].append(dates)

        ## HIGH TEMP DATA DATES ##
        #---------------------------------------------------------------
        # check how many lines exist in the list
        # if only one line, then just one date to grab
        if len(high_temp_data.splitlines()) == 1:
        
            data_dict[station][month]['high_dates'].append([filtered_high_temp_data[1]])
        
        # if there are multiple rows of high temp data, then multiple dates to grab
        elif len(high_temp_data.splitlines()) > 1:
            dates = []
            
            # grab the first date in the line
            dates.append(filtered_high_temp_data[1])
        
            # now iterate and find the rest of the dates
            for line in high_temp_data.splitlines()[1:]:
                if count_spaces(line):
                    dates.append(list(filter(None, (line.split(' '))))[0])
        
            data_dict[station][month]['high_dates'].append(dates)

    # -------------------------------------------------------------------
    y = timestamp.year
    m = timestamp.month
    d = timestamp.day
    hh = timestamp.hour
    mm = timestamp.minute
    
    api_url = f'https://mesonet.agron.iastate.edu/api/1/nwstext/{y}{m:02}{d:02}{hh:02}{mm:02}-KLZK-CXUS54-{pil}'
    print(api_url)
    
    html = urlopen(api_url).read()
    soup = BeautifulSoup(html, features="html.parser")
    
    # kill all script and style elements
    for script in soup(["script", "style"]):
        script.extract()    # rip it out
    
    # get text
    text = soup.get_text()

    # parse out the highest and lowest temp data for the month from the text
    high_temp_data = text.split('HIGHEST         ')[-1].split('LOWEST')[0]   
    low_temp_data = text.split('LOWEST           ')[-1].split('AVG. MAXIMUM')[0]  

    get_maxmin_data(high_temp_data, low_temp_data, station, m)

# ----------------------------------------------------------


y = 2023
months = [x for x in range(6)][2:]
days = [x for x in range(21)][1:]
#d = 15

#station = 'LIT'

# ----------------------------------------------------------
# establish our main working dictionary
data_dict = {}

for s in ['LIT', 'LZK', 'HRO', 'PBF']:
    _data_dict = {
        f'{s}' : {}
    }
    data_dict.update(_data_dict)

# now add the months
for key in data_dict:
    for i in range(1,13):
        _data_dict = {
             f'{calendar.month_name[i][:3].upper()}' : {# generate the abbreviated month names
            'highs': [],
            'high_dates': [], # these should still contain 12 items, but will be lists inside lists in case of multiple dates
            'lows': [],
            'low_dates': []  # these should still contain 12 items, but will be lists inside lists in case of multiple dates                                     }
                }
            }
        data_dict[key].update(_data_dict)

# ----------------------------------------------------------



# apply a broad range of days to search over for the CLM products
for m in months:

    # boolean variable to use break on for for loop
    product_found = False
    
    for d in days:

        #url = f'https://mesonet.agron.iastate.edu/wx/afos/list.phtml?by=pil&pil=CLM&day={d:02}&month={m:02}&year={y}'
        
        url = f'https://mesonet.agron.iastate.edu/wx/afos/list.phtml?source=LZK&by=pil&pil=CLM&year={y}&month={m:02}&day={d:02}'
        
        print(url)
        
        html = urlopen(url).read()
        soup = BeautifulSoup(html, features="html.parser")
        
        # kill all script and style elements
        for script in soup(["script", "style"]):
            script.extract()    # rip it out
        
        #print(text)
        
        # get text
        text = soup.get_text()

        if 'KLZK' in text.split('Skip to cccc:')[-1]:

            # convert product_found variable to True
            product_found = True
            
            text = text.split('Skip to cccc:')[-1].split('College of Ag\n')[0]

            # declare a new dictionary    
            # station_time_dict = {
            #     'station' : ['LIT', 'LZK', 'HRO', 'PBF'],
            #     'product_time' : []
            # }


            # now lets iterate through the four PILs
            for station in ['LIT', 'LZK', 'HRO', 'PBF']:
                if f'CLM{station}' in text:

                    product_time = text.split(f'CLM{station}')[-1].split('@')[-1].split('\n')[0]

                    # parse out the hour and minute from the product_time variable
                    hh = datetime.strptime(product_time, '%H:%M').hour
                    mm = datetime.strptime(product_time, '%H:%M').minute

                    # set up a timestamp object
                    timestamp = parse_timestamp(y, m, d, hh, mm)

                    text_parser(timestamp, f'CLM{station}')

                    #print(
                    #station_time_dict['product_time'].append(product_time)
                    #print(station, time)

            #print(station_time_dict)

        # exit the for loop for iterating through days and move onto the next month in the parent for loop
        if product_found:
            break

        # if we haven't found what we're looking for, then continue onto the next day in the iteration
        else:
            continue




https://mesonet.agron.iastate.edu/wx/afos/list.phtml?source=LZK&by=pil&pil=CLM&year=2023&month=02&day=01
https://mesonet.agron.iastate.edu/wx/afos/list.phtml?source=LZK&by=pil&pil=CLM&year=2023&month=02&day=02
https://mesonet.agron.iastate.edu/wx/afos/list.phtml?source=LZK&by=pil&pil=CLM&year=2023&month=02&day=03
https://mesonet.agron.iastate.edu/wx/afos/list.phtml?source=LZK&by=pil&pil=CLM&year=2023&month=02&day=04
https://mesonet.agron.iastate.edu/wx/afos/list.phtml?source=LZK&by=pil&pil=CLM&year=2023&month=02&day=05
https://mesonet.agron.iastate.edu/api/1/nwstext/202302051153-KLZK-CXUS54-CLMLIT
  77   01/11         50      27       72  01/13        

 25   01/14         31      -6       17  01/22        

https://mesonet.agron.iastate.edu/api/1/nwstext/202302051153-KLZK-CXUS54-CLMLZK
  72   01/11         49      23       68  01/01        

 24   01/31         31      -7       13  01/21        
                       01/14                                           

https://mesonet.a

HTTPError: HTTP Error 404: Not Found

In [ ]:
import requests
from urllib.request import urlopen
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import numpy.ma as ma
import os, sys
import itertools
from datetime import datetime, timezone
import calendar
import json
from operator import itemgetter
import textwrapper

In [433]:
'''
Working here, 08/02/2024

We made a function that will fetch the API links of CLM products, and save us some headache of generating those by
parsing the raw NWS text product web page text. Now with the API links, we can iterate through the links, and generate
the data and text files for the CLA products, iterating for each station/pil.

'''
# ----------------------------------------------------------
# Helping Functions
# ----------------------------------------------------------

def get_product_links(pil, y):

    '''
    A function that will query the IEM API for CLM products, and return the corresponding API links to those products

    pil : str, 6 character pil for CLM products, e.g., CLMLIT
    y : int, year in YYYY format
    
    returns:
    api_links : list, a list of API link strings
    
    '''

    print(f'Searching for {y} {pil} products...\n')
    
    # declare year and months to query
    # for a year in review, need to get Feb (Jan) to Dec (Nov), then Jan (Dec) of the new year, to complete one calendar year
    
    #y = datetime.now().year
    months = [x for x in range(13)][2:] # start in feb for jan CLM product
    days = [x for x in range(15)][1:]

    
    # empty list to store api_links for each monthly product
    api_links = []
    
    for m in months:
        print(f'---{calendar.month_name[m][:3]} {y}---')
        for d in days:

            # this is the api link that returns a json file with info, either including product issuance info, or little info, in json format
            #url = 'https://mesonet.agron.iastate.edu/api/1/nws/afos/list.json?cccc=KLZK&pil=CLMLIT&date=2023-02-04'
            url = f'https://mesonet.agron.iastate.edu/api/1/nws/afos/list.json?cccc=KLZK&pil={pil}&date={y}-{m:02}-{d:02}'
            
            # A GET request to the API
            response = requests.get(url)

            # turn the request into a json, and subsequently a dictionary
            response = response.json()
    
            if not response['data']:
                print(f'   -No {pil} product issued on {m:02}/{d:02}/{y}')
                continue
    
            elif response['data']:
                print(f'   -{pil} Product issued on {m:02}/{d:02}/{y}\n')
                api_links.append(response['data'][0]['text_link'])
                break

    # ------------------------------------------------------------
    # small section to do dec, because the dec CLM product is issued in jan of the following year
    m = 1
    print(f'---{calendar.month_name[m][:3]} {y}---')
    for d in days:

        url = f'https://mesonet.agron.iastate.edu/api/1/nws/afos/list.json?cccc=KLZK&pil={pil}&date={y+1}-{m:02}-{d:02}'
        
        # A GET request to the API
        response = requests.get(url)

        # turn the request into a json, and subsequently a dictionary
        response = response.json()

        if not response['data']:
            print(f'   -No {pil} product issued on {m:02}/{d:02}/{y}')
            continue

        elif response['data']:
            print(f'   -{pil} Product issued on {m:02}/{d:02}/{y}\n')
            api_links.append(response['data'][0]['text_link'])
            break

    print(f'Finished searching for {y} {pil} products...\n')
    return api_links

# ----------------------------------------------------------
# def count_spaces(line):

#     # this should be the date in the line
#     split_date = list(filter(None, (line.split(' '))))[0]

#     # this is the number of spaces for it to be a date occurred that month
#     if line.split(split_date)[0].count(' ') < 30:
#         return True

#     else:
#         return False
        
# ----------------------------------------------------------

def parse_timestamp(url):

    '''
    This function will take a url string, and create a datetime object from the timestamp in the url.

    url : str, the API url with the timestamp in the url

    returns:
    timestamp : a datetime object with the url timestamp
    station : str, the three letter station ID

    '''
    
    timestamp = url.split('nwstext/')[-1].split('-KLZK')[0]
    
    y = int(timestamp[:4])
    m = int(timestamp[4:6])
    # if the month of the timestamp is january, then this is the dec product
    if m == 1:
        m = 12
    # otherwise, apply the standard correction of one month subtracted
    else:
        m = m-1
    d = int(timestamp[6:8])
    hh = int(timestamp[8:10])
    mm = int(timestamp[10:])

    station = url.split('CLM')[-1]
    
    return datetime(y, m , d, hh, mm), station


# ----------------------------------------------------------


def parse_clm_text(url):

    '''
    This function will parse the API url text for climate data, and return the unfiltered text info to keys in a dictionary.

    url : str, the API url

    returns:
    text_dict : dictionary, the dictionary with all unfiltered climate text info

    '''

    
    html = urlopen(url).read()
    soup = BeautifulSoup(html, features="html.parser")
    
    # kill all script and style elements
    for script in soup(["script", "style"]):
        script.extract()    # rip it out
    
    # get text
    text = soup.get_text()

    # parse out the highest and lowest temp data for the month from the text
    high_temp_data = text.split('HIGHEST         ')[-1].split('LOWEST')[0]   
    low_temp_data = text.split('LOWEST           ')[-1].split('AVG. MAXIMUM')[0]  

    # parse out the avg high and low temp data, and the avg monthly temp data
    avg_high_temp_data = text.split('AVG. MAXIMUM  ')[-1].split('\nAVG. MINIMUM  ')[0]   
    avg_low_temp_data = text.split('AVG. MINIMUM  ')[-1].split('\nMEAN')[0]  
    avg_monthly_data = text.split('MEAN  ')[-1].split('\nDAYS MAX >= 90')[0]

    # parse out the precip and snow data
    monthly_precip_data = text.split('SNOWFALL (INCHES)')[0].split('PRECIPITATION (INCHES)')[-1].split('\nTOTALS')[-1]
    monthly_snow_data = text.split('SNOWFALL (INCHES)')[-1].split('\nTOTALS')[-1].split('\nDEGREE DAYS')[0]

    values = [high_temp_data, low_temp_data, 
                avg_high_temp_data, avg_low_temp_data, avg_monthly_data, 
                monthly_precip_data, monthly_snow_data]

    keys = ['high_temp_data_text', 'low_temp_data_text',
            'avg_high_temp_data_text', 'avg_low_temp_data_text', 'avg_monthly_data_text',
            'monthly_precip_data_text', 'monthly_snow_data_text']
    
    text_dict = dict(zip(keys, values)) 

    
    return text_dict

# ----------------------------------------------------------

def get_temp_data(high_temp_data, low_temp_data, avg_high_temp_data, avg_low_temp_data, avg_monthly_data, month):

    '''
    Worker function to extract high and low data/dates from the CLM products.

    high_temp_data : str, a parsed string that contains the monthly high temp data from the CLM
    low_temp_data : str, a parsed string that contains the monthly low temp data from the CLM
    avg_high_temp_data : str, a parsed string that contains the avg monthly high temp data from the CLM
    avg_low_temp_data : str, a parsed string that contains the avg monthly low temp data from the CLM
    avg_monthly_data : str, a parsed string that contains the avg monthly mean temp data from the CLM
    (no longer needed) station : str, the three character station ID for each climate site
    month : int, the numerical month number, can be obtained from the timestamp variable

    returns:
    None
    
    '''
    # Helper functions
    # --------------------------------------------------------
    def _count_spaces(line):

        # this should be the date in the line
        split_date = list(filter(None, (line.split(' '))))[0]
    
        # this is the number of spaces for it to be a date occurred that month
        if line.split(split_date)[0].count(' ') < 30:
            return True
    
        else:
            return False

    # --------------------------------------------------------

    
    month = calendar.month_name[month][:3].upper()
    
    # first filter the line of temp data
    filtered_low_temp_data = list(filter(None, low_temp_data.split(' '))) # low temp data
    filtered_high_temp_data = list(filter(None, high_temp_data.split(' '))) # high temp data
    
    # lets add the high and low values to the dictionary
    data_dict[month]['monthly_max_min'].append(int(filtered_high_temp_data[0]))
    data_dict[month]['monthly_max_min'].append(int(filtered_low_temp_data[0]))

    # now add the monthly avg high, low, and avg temps, and dfn
    data_dict[month]['monthly_avg_temps_dfn'].append(float(list(filter(None, (avg_high_temp_data.split(' '))))[0])) # monthly avg high
    data_dict[month]['monthly_avg_temps_dfn'].append(float(list(filter(None, (avg_low_temp_data.split(' '))))[0])) # monthly avg low
    data_dict[month]['monthly_avg_temps_dfn'].append(float(list(filter(None, (avg_monthly_data.split(' '))))[0])) # monthly avg mean temp
    data_dict[month]['monthly_avg_temps_dfn'].append(float(list(filter(None, (avg_monthly_data.split(' '))))[2])) # monthly avg mean temp dfn
    
    ## LOW TEMP DATA DATES ##
    #---------------------------------------------------------------
    # check how many lines exist in the list
    # if only one line, then just one date to grab
    if len(low_temp_data.splitlines()) == 1:
        data_dict[month].update({'low_dates' : str(filtered_low_temp_data[1])})

    # if there are multiple lines of low temp data, then check if these are the dates (occurrence dates, not records) that we want
    elif len(low_temp_data.splitlines()) > 1 and not count_spaces(low_temp_data.splitlines()[1]):
        data_dict[month].update({'low_dates' : str(filtered_low_temp_data[1])})
        
    # if there are multiple rows of low temp data, then multiple dates to grab that are valid
    else:
        # grab the first date in the line
        data_dict[month].update({'low_dates' : [filtered_low_temp_data[1]]})
        
        # now iterate and find the rest of the dates
        for line in low_temp_data.splitlines()[1:]:
            if _count_spaces(line):
                data_dict[month]['low_dates'].append(list(filter(None, (line.split(' '))))[0])


    ## HIGH TEMP DATA DATES ##
    #---------------------------------------------------------------
    # check how many lines exist in the list
    # if only one line, then just one date to grab
    if len(high_temp_data.splitlines()) == 1:
        data_dict[month].update({'high_dates' : str(filtered_high_temp_data[1])})

    # if there are multiple lines of high temp data, then check if these are the dates (occurrence dates, not records) that we want
    elif len(high_temp_data.splitlines()) > 1 and not count_spaces(high_temp_data.splitlines()[1]):
        data_dict[month].update({'high_dates' : str(filtered_high_temp_data[1])})

    # if there are multiple rows of high temp data, then multiple dates to grab
    else:
        
        # grab the first date in the line
        data_dict[month].update({'high_dates' : [filtered_high_temp_data[1]]})
    
        # now iterate and find the rest of the dates
        for line in high_temp_data.splitlines()[1:]:
            if _count_spaces(line):
                data_dict[month]['high_dates'].append(list(filter(None, (line.split(' '))))[0])



# ----------------------------------------------------------

def get_precip_data(monthly_precip_data, month):

    '''
    This function will parse the precip text info from the API url, and will append precip data
    to the main working dictionary.

    monthly_precip_data : str, the parsed monthly precip data as a string
    month : int, the month as an integer

    returns: 
    None
    
    '''
    
    # Helper Sub-function of the main function        
    # --------------------------------------------------------   
    
    def _check_record(param):
        if 'R' in str(param):
            return param.split('R')[0]
        else:
            return param

    # --------------------------------------------------------
    
    # convert the integer month to the three letter month name
    month = calendar.month_name[month][:3].upper()


    # ---- Monthly Rain Data -----
    monthly_total_rain = _check_record(list(filter(None, (monthly_precip_data.split(' '))))[0]) # monthly total rain
    monthly_rain_dfn = _check_record(list(filter(None, (monthly_precip_data.split(' '))))[2])
    
    # add the monthly precip total and dfn
    data_dict[month]['monthly_rain_and_dfn'].append(float(monthly_total_rain)) # monthly total
    data_dict[month]['monthly_rain_and_dfn'].append(float(monthly_rain_dfn)) # monthly dfn

    # now lets get the calendar max, 24 hr max values and dates for rain
    for idx, line in enumerate(monthly_precip_data.split('\nGREATEST\n')[-1].splitlines()):
        #print(list(filter(None, (line.split(' ')))))
        if idx == 0:
            max_clndr_rain = float(list(filter(None, (line.split(' '))))[3])
            max_clndr_rain_dates = " ".join(list(filter(None, (line.split(' '))))[4:])
    
        elif idx == 1:
            max_stormtotal_rain = float(list(filter(None, (line.split(' '))))[-1])
    
    # check for a different max storm total rainfall date
    lst = monthly_precip_data.split('\nGREATEST\n')[-1][:-2].splitlines()
    empty_date = '(MM/DD(HH))' # string we are searching for that designates no storm total date i.e. it equals the 24 hr total
    
    # if we find the empty_date search string, then add the 24 hr total dates to storm total dates
    if any(empty_date in x for x in lst):
        #max_stormtotal_rain_dates = " ".join(list(filter(None, (monthly_precip_data.split('\nGREATEST\n')[-1][:-2].splitlines()[0].split(' '))))[4:])
        max_stormtotal_rain_dates = max_clndr_rain_dates
        
    # if we do not find the empty_date search string, then handle adding a potentially different storm total date
    else:
        max_stormtotal_rain_dates = " ".join(list(filter(None, (monthly_precip_data.split('\nGREATEST\n')[-1][:-2].splitlines()[1].split(' '))))[3:])
        
    # 24 hour (calendar day max) rainfall data
    data_dict[month]['max_clndr_24hr_rain'].append(max_clndr_rain) # 24 hr max (calendar day max)
    data_dict[month]['max_clndr_24hr_rain'].append(max_clndr_rain_dates) # 24 hr max dates
    
    data_dict[month]['max_clndr_24hr_rain'].append(max_stormtotal_rain) # max storm total 
    data_dict[month]['max_clndr_24hr_rain'].append(max_stormtotal_rain_dates) # max storm total dates (set equal to 24 hr max for now)



def get_snow_data(monthly_snow_data, month):

    '''
    This function will parse the precip text info from the API url, and will append snow data
    to the main working dictionary.

    monthly_snow_data : str, the parsed monthly snow data as a string
    month : int, the month as an integer

    returns: 
    None
    
    '''

    # Helper Sub-functions of the main function
    # --------------------------------------------------------
    def _check_record(param):
        if 'R' in str(param):
            return param.split('R')[0]
        else:
            return param

    # --------------------------------------------------------
    def _check_missing_snow(param, month):
        try:
            data_dict[month]['monthly_snow_sdepth_dfn'].append(float(param))
        
        except ValueError:
            if 'T' in str(param):
                data_dict[month]['monthly_snow_sdepth_dfn'].append(param) 
            else:
                data_dict[month]['monthly_snow_sdepth_dfn'].append(0.0)

    # --------------------------------------------------------

    # first, if month is not in a primary cool season month, then auto fill values and end the function call
    if month not in [1, 2, 3, 4, 10, 11, 12]:

        # convert the integer month to the three letter month name
        month = calendar.month_name[month][:3].upper()
        
        # this will add [0.0, 0.0, 0.0, np.nan]
        blank_values = [0.0, 0.0, 0.0, np.nan]
        data_dict[month].update({'monthly_snow_sdepth_dfn' : blank_values})

        # for i in range(3):
        #     data_dict[month]['monthly_snow_sdepth_dfn'].append(0.0)
        # data_dict[month]['monthly_snow_sdepth_dfn'].append(np.nan)

        # this will add [0.0, np.nan]
        blank_values = [0.0, np.nan]
        data_dict[month].update({'max_clndr_24hr_snow' : blank_values})
        # data_dict[month]['max_clndr_24hr_snow'].append(0.0)
        # data_dict[month]['max_clndr_24hr_snow'].append(np.nan) 
        return
        

    # if we do have cool season month, then proceed 

    # convert the integer month to the three letter month name
    month = calendar.month_name[month][:3].upper()
    

    # ---- Monthly Snow Data ----
    # add the monthly snow total and dfn
    monthly_total_snow = _check_record(list(filter(None, (monthly_snow_data.split(' '))))[0]) # monthly total snow
    monthly_grtst_sdepth = _check_record(list(filter(None, (monthly_snow_data.split('\nGREATEST\n SNOW DEPTH')[-1].splitlines()[0].split(' '))))[0]) # monthly greatest snow depth
    monthly_snow_dfn = _check_record(list(filter(None, (monthly_snow_data.split(' '))))[2]) # monthly total snow dfn
    monthly_grtst_24hr_snow = _check_record(list(filter(None, (monthly_snow_data.split('\nGREATEST\n')[-1].split(' 24 HR TOTAL')[-1].splitlines()[0].split(' '))))[0])

    # Add the data to the main dictionary
    # for the non winter months, monthly snow may just be set to 'MM' in the CLM product
    _check_missing_snow(monthly_total_snow, month) # this will add the monthly total snow
    _check_missing_snow(monthly_snow_dfn, month) # this will add the monthly total snow dfn  
    _check_missing_snow(monthly_grtst_sdepth, month) # this will add the greatest monthly snow depth


    
    # ---- Greatest Snow Depth Dates ----
    # Now lets check for dates on greatest snow depth and 24 max total
    lst = monthly_snow_data.split('\nGREATEST\n')[-1].splitlines()
    
    # if monthly max snow depth is missing, set dates to nan
    if monthly_grtst_sdepth in ['0', 'MM']:
    #if monthly_grtst_sdepth == 'MM':
        monthly_grtst_sdepth_dates = np.nan
    
    # for greatest snow depth date (multiple dates)
    if any('/' in x for x in lst[0]) and len(lst) > 2:
    
        monthly_grtst_sdepth_dates = []
    
        # get the first date and make sure it is valid
        split_date = list(filter(None, (lst[0].split(' '))))[-1]
        if lst[0].split(split_date)[0].split(monthly_grtst_sdepth)[-1].count(' ') <= 3:
            monthly_grtst_sdepth_dates.append(split_date)
    
        # now iterate over the remainder of possible dates
        for idx, item in enumerate(lst[1:-1]): # skip the first, and the last line because the last line is the 24 hr max data
            split_date = list(filter(None, (item.split(' '))))[0]
            if lst[idx].split(split_date).count(' ') <= 25:
                monthly_grtst_sdepth_dates.append(split_date)
    
    # for greatest snow depth date (only one date)
    elif any('/' in x for x in lst[0]) and len(lst) == 2:
        
        # get the first date and make sure it is valid
        split_date = list(filter(None, (lst[0].split(' '))))[-1]
        if lst[0].split(split_date)[0].split(monthly_grtst_sdepth)[-1].count(' ') <= 3:
            monthly_grtst_sdepth_dates = split_date   
    else:
        monthly_grtst_sdepth_dates = np.nan

    
    # ---- Max 24 hr Snow Dates ----
    # lets follow similar logic here...
    
    # if monthly max 24 hr snow is missing, then set dates to nan
    if monthly_grtst_24hr_snow in ['0.0', 'MM']:
    #if monthly_grtst_24hr_snow == 'MM':
        monthly_grtst_24hr_snow_dates = np.nan
    
    # for greatest snow depth date
    elif any('/' in x for x in lst[-1]):
    
        # lets get the first occurrence of a date and use that to split
        first_date = list(filter(None, (lst[-1].split(monthly_grtst_24hr_snow)[-1].split('TO')[0].split(' '))))
        first_date = list(filter(lambda k: 'R' not in k, first_date)) # in case there is a record value for 24 hr snow, there will still be an 'R'
        
        # now lets split based on the first date and count the spaces
        if lst[-1].split(first_date[0]).count(' ') <= 4:
            monthly_grtst_24hr_snow_dates = " ".join(list(filter(None, (monthly_snow_data.split('\nGREATEST\n')[-1].splitlines()[1].split(' '))))[4:7])
        
        else:
            monthly_grtst_24hr_snow_dates = np.nan
    
    else:
        monthly_grtst_24hr_snow_dates = np.nan


    # now add the data to the main dictionary
    data_dict[month]['monthly_snow_sdepth_dfn'].append(monthly_grtst_sdepth_dates) # add the greatest snow depth date(s) to the dictionary
    
    try:
        data_dict[month]['max_clndr_24hr_snow'].append(float(monthly_grtst_24hr_snow)) # add the greatest 24 hr snow total to the dictionary
    except ValueError:
        if monthly_grtst_24hr_snow in ['0.0', 'MM']:
            data_dict[month]['max_clndr_24hr_snow'].append(0.0) # add the greatest 24 hr snow total to the dictionary 
        elif monthly_grtst_24hr_snow in ['T']:
            data_dict[month]['max_clndr_24hr_snow'].append('T') # add trace for the greatest 24 hr snow total to the dictionary
        
    data_dict[month]['max_clndr_24hr_snow'].append(monthly_grtst_24hr_snow_dates) # add the greatest 24 hr snow total dates to the dictionary    



# ----------------------------------------------------------  

    








# Main script begins here...

# -----------------------------------
# establish our main working dictionary
# data_dict = {}

# for s in ['LIT', 'LZK', 'HRO', 'PBF']:
#     _data_dict = {
#         f'{s}' : {}
#     }
#     data_dict.update(_data_dict)

# # now add the months
# for key in data_dict:
#     for i in range(1,13):
#         _data_dict = {
#             # generate the abbreviated month names
#              f'{calendar.month_name[i][:3].upper()}' : {
#             'monthly_max_min': [],
#             'high_dates': None,
#             'low_dates': None,
#             'monthly_avg_temps_dfn' : [], # avg high, low, mean, mean dfn
#             'monthly_rain_and_dfn' : [], # monthly total rain and dfn
#             'max_clndr_24hr_rain' : [],
#             'monthly_snow_sdepth_dfn' : [], # monthly total rain and dfn
#             'max_clndr_24hr_snow' : []
#                 }
#             }
#         data_dict[key].update(_data_dict)

# -----------------------------------
# using this for dictionary structure instead, 
# only going to focus on one station at a time for each call of the main() function

# establish our main working dictionary
data_dict = {}

for i in range(1,13):
    _data_dict = {f'{calendar.month_name[i][:3].upper()}' : {}}
    data_dict.update(_data_dict)


# now add the months
for key in data_dict:
    _data_dict = {
        # generate the abbreviated month names
         #f'{calendar.month_name[i][:3].upper()}' : {
        'monthly_max_min': [],
        'high_dates': None,
        'low_dates': None,
        'monthly_avg_temps_dfn' : [], # avg high, low, mean, mean dfn
        'monthly_rain_and_dfn' : [], # monthly total rain and dfn
        'max_clndr_24hr_rain' : [], # [max calendar day rainfall (float), dates, max 24 hr rainfall (float), dates]
        'monthly_snow_sdepth_dfn' : [], # monthly total snow, total snow dfn, greatest snow depth, and date(s) of greatest snow depth
        'max_clndr_24hr_snow' : [] # max 24 hr total snow and date(s) of 24 hr total snow
        }
    
    data_dict[key].update(_data_dict)




# keep this here to only have to run for api links when necessary, this is time consuming
# first lets get the API links for a station
#api_links = get_product_links(', year) # these variables will be replaced with place holders in the main function

# -----------------------------------


def construct_data_dict(pil, year):
    
    # first lets get the API links for a station
    #api_links = get_product_links(pil, year) # these variables will be replaced with place holders in the main function
    
    
    for url in api_links:
    
        timestamp, station = parse_timestamp(url) # get the datetime and station from the url string
        #high_temp_data, low_temp_data = parse_clm_text(url) # this will get the high temp and low temp data from the text within the url
        text_dict = parse_clm_text(url) # this will get the high temp and low temp data from the text within the url
    
        # parse the high/low temp and add it to the main data dictionary
        get_temp_data(text_dict['high_temp_data_text'], text_dict['low_temp_data_text'], 
                        text_dict['avg_high_temp_data_text'], text_dict['avg_low_temp_data_text'], text_dict['avg_monthly_data_text'],
                        timestamp.month)

        # parse the precip data and add it to the main data dictionary
        get_precip_data(text_dict['monthly_precip_data_text'], timestamp.month)
    
        # parse the snow data and add it to the main data dictionary
        get_snow_data(text_dict['monthly_snow_data_text'], timestamp.month)

def main(pil, year):
    construct_data_dict(pil, year)

main('CLMLZK', 2021)

#data_dict['LZK']


'''
Notes, this all *appears* to be working, will need to test it out over some years and sites...

'''


0
14
0
0
0
0
0


'\nNotes, this all *appears* to be working, will need to test it out over some years and sites...\n\n'

In [415]:
data_dict

{'JAN': {'monthly_max_min': [73, 24],
  'high_dates': '01/25',
  'low_dates': '01/12',
  'monthly_avg_temps_dfn': [49.8, 34.3, 42.1, 1.0],
  'monthly_rain_and_dfn': [2.74, -0.68],
  'max_clndr_24hr_rain': [0.78, '01/24 TO 01/24', 0.78, '01/24 TO 01/24'],
  'monthly_snow_sdepth_dfn': ['T', -1.7, 0.0, nan],
  'max_clndr_24hr_snow': ['T', '01/15 TO 01/15']},
 'FEB': {'monthly_max_min': [73, -1],
  'high_dates': ['02/23', '02/28'],
  'low_dates': '02/16',
  'monthly_avg_temps_dfn': [42.9, 27.8, 35.4, -9.9],
  'monthly_rain_and_dfn': [5.23, 1.57],
  'max_clndr_24hr_rain': [1.81, '02/28 TO 02/28', 1.81, '02/28 TO 02/28'],
  'monthly_snow_sdepth_dfn': [18.7, 16.9, 14.0, '02/18'],
  'max_clndr_24hr_snow': ['10.9', '02/14 TO 02/15']},
 'MAR': {'monthly_max_min': [80, 33],
  'high_dates': '03/27',
  'low_dates': '03/03',
  'monthly_avg_temps_dfn': [66.2, 46.9, 56.5, 2.6],
  'monthly_rain_and_dfn': [5.98, 1.21],
  'max_clndr_24hr_rain': [2.36, '03/17 TO 03/17', 2.36, '03/17 TO 03/17'],
  'monthly

In [50]:
api_links = get_product_links('CLMLZK', 2021) # these variables will be replaced with place holders in the main function

Searching for 2021 CLMLZK products...

---Feb 2021---
   -No CLMLZK product issued on 02/01/2021
   -CLMLZK Product issued on 02/02/2021

---Mar 2021---
   -No CLMLZK product issued on 03/01/2021
   -No CLMLZK product issued on 03/02/2021
   -No CLMLZK product issued on 03/03/2021
   -No CLMLZK product issued on 03/04/2021
   -CLMLZK Product issued on 03/05/2021

---Apr 2021---
   -No CLMLZK product issued on 04/01/2021
   -CLMLZK Product issued on 04/02/2021

---May 2021---
   -CLMLZK Product issued on 05/01/2021

---Jun 2021---
   -CLMLZK Product issued on 06/01/2021

---Jul 2021---
   -No CLMLZK product issued on 07/01/2021
   -CLMLZK Product issued on 07/02/2021

---Aug 2021---
   -CLMLZK Product issued on 08/01/2021

---Sep 2021---
   -CLMLZK Product issued on 09/01/2021

---Oct 2021---
   -No CLMLZK product issued on 10/01/2021
   -CLMLZK Product issued on 10/02/2021

---Nov 2021---
   -No CLMLZK product issued on 11/01/2021
   -CLMLZK Product issued on 11/02/2021

---Dec 2021---

In [193]:
#url = 'https://mesonet.agron.iastate.edu/api/1/nwstext/202302051153-KLZK-CXUS54-CLMLZK'
url = 'https://mesonet.agron.iastate.edu/api/1/nwstext/202103050058-KLZK-CXUS54-CLMLZK'
#url = 'https://mesonet.agron.iastate.edu/api/1/nwstext/202303041555-KLZK-CXUS54-CLMHRO' # multiple dates of greatest snow depth

html = urlopen(url).read()
soup = BeautifulSoup(html, features="html.parser")

# kill all script and style elements
for script in soup(["script", "style"]):
    script.extract()    # rip it out

# get text
text = soup.get_text()

avg_high_temp_data = text.split('AVG. MAXIMUM  ')[-1].split('\nAVG. MINIMUM  ')[0]   
avg_low_temp_data = text.split('AVG. MINIMUM  ')[-1].split('\nMEAN')[0]  
avg_monthly_data = text.split('MEAN  ')[-1].split('\nDAYS MAX >= 90')[0]

monthly_precip_data = text.split('SNOWFALL (INCHES)')[0].split('PRECIPITATION (INCHES)')[-1].split('\nTOTALS')[-1]
monthly_snow_data = text.split('SNOWFALL (INCHES)')[-1].split('\nTOTALS')[-1].split('\nDEGREE DAYS')[0]

monthly_snow_data.splitlines()

['          18.7R               1.8    16.9      0.3               ',
 ' LIQUID EQUIV   1.90               0.20    1.70     0.03               ',
 'SINCE 7/1       18.8                4.0    14.8       MM               ',
 ' LIQUID 7/1     1.88               0.00    1.88       MM               ',
 'SNOWDEPTH AVG.     2                                                   ',
 'DAYS >= TRACE      6                1.2     4.8        3               ',
 'DAYS >= 1.0        3                                   0               ',
 'GREATEST',
 ' SNOW DEPTH       14R  02/18                                           ',
 ' 24 HR TOTAL    10.9R  02/14 TO 02/15                0.3  02/20 TO 02/20']

In [461]:
'''
Alrighty, now we have a script that will make us a dictionary... 
Lets try to write the data to tables in a text file...

'''


# --------------------------------------------------------------------------------------------
def _make_temp_table(f, year, stn_name):

    '''
    This function will write in the annual temperature summary table to the supplemental CLA product text file.

    f : file, the text file that is currently open that is being written to
    year : int, the climate year
    stn_name : str, the three letter climate station ID

    returns:
    None

    '''

    
    # put together the data for iterating quickly
    months = [calendar.month_name[i][:3].upper() for i in range(1,13)]
    
    avg_high_temps = [data_dict[m]['monthly_avg_temps_dfn'][0] for m in months]
    avg_low_temps = [data_dict[m]['monthly_avg_temps_dfn'][1] for m in months]
    avg_temps = [data_dict[m]['monthly_avg_temps_dfn'][2] for m in months]
    avg_temps_dfn = [data_dict[m]['monthly_avg_temps_dfn'][3] for m in months]
    
    high_temps = [data_dict[m]['monthly_max_min'][0] for m in months]
    high_temp_dates = [data_dict[m]['high_dates'] for m in months]
    
    low_temps = [data_dict[m]['monthly_max_min'][1] for m in months]
    low_temp_dates = [data_dict[m]['low_dates'] for m in months]

    # now lets write the table into the text file
    f.write(f'\n{year} TEMPERATURE AVERAGES AND EXTREMES                     {stn_name}, ARKANSAS\n')
    f.write(table_sep)
    f.write('             AVERAGE TEMPERATURES             |            TEMPERATURE EXTREMES\n')
    f.write('MONTH     HIGH     LOW     MONTHLY     DFN    |    MAX      DATE(S)         MIN      DATE(S)\n')
    f.write(table_sep)

    # order is as follows: month, avg max, avg min, avg mean, avg dfn, max, max dates, min, min dates
    for m, avgmx, avgmn, avgt, dfn, mx, mxdt, mn, mndt in zip(
                                                              months, 
                                                              avg_high_temps, avg_low_temps, avg_temps, avg_temps_dfn,
                                                              high_temps, high_temp_dates,
                                                              low_temps, low_temp_dates
                                                              ):
        # do some formatting here...
        # start with constants for certain spaces in the sequence
        space1 = space*4 # the space between the dfn, and the | separator
        space2 = space*7 # the space between the monthly high temp, and high temp dates
        space3 = space*14 # the space b/w the max temp dates, and the monthly min temp
        space4 = space*7 # the space b/w the min temp, and min temp dates

        # format the dfn spacing and positive dfn's
        if dfn >= 10.0 or dfn <= -10.0:
            space1 = space*3

        if dfn > 0.0:
            dfn = f'+{dfn}'

        # format the max temp spacing if > 100
        if mx >= 100:
            space2 = space*6

        # format the min temp spacing if < -10
        if mn <= -10:
            space4 = space*6
        
        # format the max temp dates
        if isinstance(mxdt, list):
            adj = (len(mxdt)*3)-3
            space3 = space*(14-adj)
            mxdt = [i.split('/')[-1] for i in mxdt]
            mxdt = "/".join(sorted(mxdt, key = int)) # sort the days in ascending order
        
        else:
            mxdt = mxdt.split('/')[-1]

        # format the min temp dates
        if isinstance(mndt, list):
            mndt = [i.split('/')[-1] for i in mndt]
            mndt = "/".join(sorted(mndt, key = int)) # sort the days in ascending order
        else:
            mndt = mndt.split('/')[-1]                    
        
        f.write(f'{m}       {avgmx}    {avgmn}     {avgt}       {dfn}{space1}|    {mx}{space2}{mxdt}{space3}{mn}{space4}{mndt}\n')


    # -------------------------------------------------
    # next add in the annual summary data
    f.write(table_sep) # add a table separator

    # constant space values we will need to adjust 
    space1 = space*4 # space b/w avg dfn and the | separator
    space2 = space*7 # space b/w the yearly max temp and the yearly max temp dates 
    space3 = space*10 # space b/w the yearly max temp dates and the yearly min
    space4 = space*7 # space b/w the yearly min and yearly min dates
    
    yrly_avgmx = np.round(np.mean(avg_high_temps),1)
    yrly_avgmn = np.round(np.mean(avg_low_temps),1)
    yrly_avg = np.round(np.mean(avg_temps),1)
    
    yrly_avg_dfn = np.round(np.sum(avg_temps_dfn), 1)
    if yrly_avg_dfn >= 10.0 or yrly_avg_dfn <= -10.0:
        space1 = space*3

    if yrly_avg_dfn > 0.0:
        yrly_avg_dfn = f'+{yrly_avg_dfn}'
    
    yrly_mx = np.max(high_temps)
    if yrly_mx >= 100.0:
        space2 = space*6
        
    yrly_mn = np.min(low_temps)
    if yrly_mn <= -10.0:
        space4 = space*6

    # -------------------------------------------------
    def _find_annual_extreme_dates(temp_dates_lst, idx, max_temp = False):

        '''
        This function will parse a temp dates list with a given index and return a string of the extreme dates

        temp_dates_lst : list, a list of the extreme temperature dates for a month
        idx : int, the index value of the extreme temperature date for the year
        max_temp : bool, default = False, if max_temp, then edit space3 variable if necessary

        returns:
        dt : str, a string of the formatted extreme temp dates
        _space3 : str, a string of formatted spaces for the space 3 variable in the annual values line
    
        '''
        
        _space3 = space3 # we only return this for the max temp dates
        
        if isinstance(temp_dates_lst[idx], list):
            if max_temp:
                adj = (len(temp_dates_lst[idx])*3)-3
                _space3 = space*(10-adj)
            
            dt = [i.split('/')[-1] for i in temp_dates_lst[idx]]
            dt = "/".join(sorted(dt, key = int)), # sort the days in ascending order
        else:
            dt = temp_dates_lst[idx].split('/')[-1]

        return dt, _space3

    # -------------------------------------------------

    # format the annual max temp date(s)
    yrly_mx = np.max(high_temps)
    idx = high_temps.index(yrly_mx)
    
    mxdt, space3 = _find_annual_extreme_dates(high_temp_dates, idx, max_temp = True)
    yrly_mxdt = f'{calendar.month_name[idx+1][:3].upper()} {mxdt}'

    
    # format the annual min temp date(s)
    yrly_mn = np.min(low_temps)
    idx = low_temps.index(yrly_mn)

    mndt, _ = _find_annual_extreme_dates(low_temp_dates, idx, max_temp = False)
    yrly_mndt = f'{calendar.month_name[idx+1][:3].upper()} {mndt}'

    # now lets write in the data
    f.write(f'ANNUAL    {yrly_avgmx}    {yrly_avgmn}     {yrly_avg}       {yrly_avg_dfn}{space1}|    {yrly_mx}{space2}{yrly_mxdt}{space3}{yrly_mn}{space4}{yrly_mndt}\n')

# --------------------------------------------------------------------------------------------

def _make_precip_table(f, year, stn_name):

    '''
    This function will write in the annual rain summary table to the supplemental CLA product text file.

    f : file, the text file that is currently open that is being written to
    year : int, the climate year
    stn_name : str, the three letter climate station ID

    returns:
    None

    '''

    # --------------------------------------------
    def _find_numeric_suffix(myDate):
        
            '''
            This function will take a string date, formatted as 'nn', e.g. '05', and assign a suffix based on the number.
        
            myDate : str, a date number, formatted as 'nn', e.g. '05'
        
            returns:
            myDate : str, formatted as 'nnTH', 'nnST', 'nnND', 'nnRD'
        
            '''
            
            date_suffix = ["TH", "ST", "ND", "RD"]
        
            if int(myDate) % 10 in [1, 2, 3] and int(myDate) not in [11, 12, 13]:
                return f'{myDate}{date_suffix[int(myDate) % 10]}'
            else:
                return f'{myDate}{date_suffix[0]}'


    # --------------------------------------------
    # add table headers and column information
    
    f.write(f'\n\n{year} RAINFALL, DEPARTURES, AND EXTREMES                    {stn_name}, ARKANSAS\n')
    f.write(table_sep)
    f.write('MONTH        RAINFALL       DFN               MAX/CALENDAR DAY            MAX/24 HOURS\n')
    f.write(table_sep)

    # --------------------------------------------
    # now lets compile and organize the data
    # put together the data for iterating quickly
    months = [calendar.month_name[i][:3].upper() for i in range(1,13)]

    precip = [data_dict[m]['monthly_rain_and_dfn'][0] for m in months]
    precip_dfn = [data_dict[m]['monthly_rain_and_dfn'][1] for m in months]
    
    precip_mx_clndr = [data_dict[m]['max_clndr_24hr_rain'][0] for m in months]
    precip_mx_clndr_dates = [data_dict[m]['max_clndr_24hr_rain'][1] for m in months]

    precip_mx_storm_total = [data_dict[m]['max_clndr_24hr_rain'][2] for m in months]
    precip_mx_storm_total_dates = [data_dict[m]['max_clndr_24hr_rain'][3] for m in months]

    # for loop begins
    for m, pcp, dfn, pcp_mx_clndr, pcp_mx_clndr_dates, pcp_mx_storm_total, pcp_mx_storm_total_dates in zip(
                                                            months,
                                                            precip, precip_dfn,
                                                            precip_mx_clndr, precip_mx_clndr_dates,
                                                            precip_mx_storm_total, precip_mx_storm_total_dates
                                                            ):
        # --------------------------------------------
        # lets do some formatting here
        # start with constants for certain spaces in the sequence
        space1 = space*10 # the space b/w the precip and the departure
        space2 = space*14 # the space b/w the precip dfn and the precip calendar day max
        space3 = space*19 # space b/w precip calendar day max date and the max precip storm total

        # edit space 1 if monthly precip is greater than 10 inches
        if pcp >= 10.00:
            space1 = space*9

        # edit space 2 if monthly precip dfn is >= 10.0 or <= -10.0
        if dfn >= 10.0 or dfn <= -10.0:
            space2 = space*12 

        # assign a sign to dfn if it is positive
        if dfn >= 0.0:
            dfn = f'+{dfn:.2f}'
            
        # --------------------------------------------
        # lets format the max calendar day rainfall dates here...
        #for date in pcp_mx_clndr_dates:
        bdt = list(filter(None, (pcp_mx_clndr_dates.split('TO')[0].split(' '))))[0]
        edt = list(filter(None, (pcp_mx_clndr_dates.split('TO')[-1].split(' '))))[0]
    
        # if the dates are the same, then format for one day
        if bdt == edt:
            clndr_mxdt = _find_numeric_suffix(bdt.split('/')[-1])

        # if the dates are not the same, then format for multiple dates
        elif bdt != edt:
            bdt = _find_numeric_suffix(bdt.split('/')[-1])
            edt = _find_numeric_suffix(edt.split('/')[-1])
            clndr_mxdt = f'{bdt}-{edt}' 
            space3 = space*14

        # --------------------------------------------
        # lets format the max storm total rainfall dates here...
        bdt = list(filter(None, (pcp_mx_storm_total_dates.split('TO')[0].split(' '))))[0]
        edt = list(filter(None, (pcp_mx_storm_total_dates.split('TO')[-1].split(' '))))[0]
    
        # if the dates are the same, then format for one day
        if bdt == edt:
            stormtotal_mxdt = _find_numeric_suffix(bdt.split('/')[-1])
            
        # if the dates are not the same, then format for multiple dates
        elif bdt != edt:
            bdt = _find_numeric_suffix(bdt.split('/')[-1])
            edt = _find_numeric_suffix(edt.split('/')[-1])
            stormtotal_mxdt = f'{bdt}-{edt}'

        # write in the line of monthly data
        f.write(f'{m}          {pcp:.2f}{space1}{dfn}{space2}{pcp_mx_clndr:.2f}/{clndr_mxdt}{space3}{pcp_mx_storm_total:.2f}/{stormtotal_mxdt}\n')
        # end of for loop

    # --------------------------------------------
    # constant space values that may be edited based on the data
    space1 = space*14 # space b/w yearly dfn, and yrly calendar day max precip
    space2 = space*17 # space b/w yearly calendar day max precip date, and yearly daily max storm total precip value
    
    # now lets add the annual data
    yrly_rain = np.round(np.sum(precip),2)
    yrly_dfn = np.round(np.sum(precip_dfn),2)

    # adjust space1 if yearly precip dfn is >= 10.0 or <= -10.0
    if yrly_dfn >= 10.0 or yrly_dfn <= -10.0:
        space1 = space*13
    
    # if the yrly precip dfn is >= 0.0, then assign a positive sign in the string
    if yrly_dfn >= 0.0:
        yrly_dfn = f'+{yrly_dfn}'

    # get the max calendar day and storm total precip values
    # mx_clndr_pcp = np.max(precip_mx_clndr)
    # mx_stormtotal_pcp = np.max(precip_mx_storm_total)

    # need to format the annual precip extreme dates...

    # -------------------------------------------------
    def _find_annual_extreme_dates(pcp_dates_lst, idx, clndr_day_max = False): 
    
        '''
        This function will parse a precip dates list with a given index and return a string of the extreme dates
        
        pcp_dates_lst : list, a list of the extreme precip dates for a month
        idx : int, the index value of the extreme precip date for the year
        clndr_day_max : bool, default = False, if clndr_day_max, then edit space2 variable if necessary
        
        returns:
        yrly_mxdt : str, a string of the formatted extreme precip dates
        _space2 : str, a string of formatted spaces for the space 2 variable in the annual values line
        
        '''
        _space2 = space2 # we only edit this for the calendar day max precip date, if it is two dates
        
        bdt = list(filter(None, (pcp_dates_lst[idx].split('TO')[0].split(' '))))[0]
        edt = list(filter(None, (pcp_dates_lst[idx].split('TO')[-1].split(' '))))[0]
        
        if bdt == edt:
            yrly_mxdt = bdt.split('/')[-1]
        
        elif bdt != edt:
            bdt = bdt.split('/')[-1]
            edt = edt.split('/')[-1]
            yrly_mxdt = f'{bdt}-{edt}'
            
            if clndr_day_max:
                _space2 = space*14
                
        return yrly_mxdt, _space2

    # -------------------------------------------------

    # yearly calendar day max precip date
    clndr_pcp_mx = np.max(precip_mx_clndr)
    idx = precip_mx_clndr.index(clndr_pcp_mx) # index value of the calendar day max precip

    clndr_max_pcp_dt, space2 = _find_annual_extreme_dates(precip_mx_clndr_dates, idx, clndr_day_max = True)
    yrly_clndr_pcp_mx = f'{calendar.month_name[idx+1][:3].upper()} {clndr_max_pcp_dt}'

    # -------------------------------------------------
    
    # yearly daily storm total max precip date
    storm_total_pcp_mx = np.max(precip_mx_storm_total)
    idx = precip_mx_storm_total.index(storm_total_pcp_mx)
    
    storm_total_pcp_max_dt, _ = _find_annual_extreme_dates(precip_mx_storm_total_dates, idx, clndr_day_max = False)
    yrly_storm_total_pcp_max_dt = f'{calendar.month_name[idx+1][:3].upper()} {storm_total_pcp_max_dt}'

    # -------------------------------------------------
    # write in the annual precip summary data
    f.write(table_sep)
    f.write(f'ANNUAL       {yrly_rain}         {yrly_dfn}{space1}{clndr_pcp_mx}/{yrly_clndr_pcp_mx}{space2}{storm_total_pcp_mx}/{yrly_storm_total_pcp_max_dt}\n')

# --------------------------------------------------------------------------------------------

def _make_snow_table(f, year, stn_name):

    '''
    This function will write in the annual snow summary table to the supplemental CLA product text file.

    f : file, the text file that is currently open that is being written to
    year : int, the climate year
    stn_name : str, the three letter climate station ID

    returns:
    None

    '''

    # --------------------------------------------
    # add table headers and column information

    f.write(f'\n\n{year} SNOWFALL, DEPARTURES, AND EXTREMES                    {stn_name}, ARKANSAS\n')
    f.write(table_sep)
    f.write('MONTH   SNOW      DFN       MAX/CALENDAR DAY       MAX/24 HOUR       GREATEST DEPTH/DATE\n')
    f.write(table_sep)
    
    # --------------------------------------------
    # now lets compile and organize the data
    # put together the data for iterating quickly
    #months = [calendar.month_name[i][:3].upper() for i in range(1,13)]
    snow_months = ['JAN', 'FEB', 'MAR', 'APR', 'OCT', 'NOV', 'DEC'] # these are the months we need for the snow table summary
    
    # we need to parse the months real quickly to just snow months
    #months = [m for m in months if m in snow_months]

    snow = [data_dict[m]['monthly_snow_sdepth_dfn'][0] for m in months if m in snow_months]
    snow_dfn = [data_dict[m]['monthly_snow_sdepth_dfn'][1] for m in months if m in snow_months]
    
    mx_sdepth = [data_dict[m]['monthly_snow_sdepth_dfn'][2] for m in months if m in snow_months]
    mx_sdepth_dt = [data_dict[m]['monthly_snow_sdepth_dfn'][3] for m in months if m in snow_months]
    
    mx_24hr_snow = [data_dict[m]['max_clndr_24hr_snow'][0] for m in months if m in snow_months]
    mx_24hr_snow_dt = [data_dict[m]['max_clndr_24hr_snow'][1] for m in months if m in snow_months]    

    # for loop begins
    for m, sn, dfn, mx_sd, mx_sd_dt, mx_24hr_sn, mx_24hr_sn_dt in zip(
                                                                snow_months,
                                                                snow, snow_dfn,
                                                                mx_sdepth, mx_sdepth_dt,
                                                                mx_24hr_snow, mx_24hr_snow_dt
                                                                  ):
        # constant space values that may need to be edited
        space1 = space*6 # space b/w the monthly snow, and the dfn
        space2 = space*7 # space b/w the dfn and the calendar day max snow/date
        space3 = space*20 # space b/w the calendar day max snow/date and the 24 hr max snow date
        space4 = space*15 # space b/w 24 hour max, and 
        
        # lets do some formatting here
        # account for when there is trace monthly snow
        if sn == 'T':
            sn = 'T  '
        
        # edit space1 when snow is >= 10.0
        if isinstance(sn, float) and sn >= 10.0: # used isinstance to keep value error from popping up when making comparison
            space1 = space*5

        # adjust dfn parameters
        if dfn >= 10.0: # snow dfn will never less than 10 inches for our climate sites
            dfn = f'+{dfn}'
            space2 = space*6
        
        elif dfn == 0.0:
            dfn = f' {dfn}'

        # --------------------------------------------
        # lets format the max calendar day and 24 hr snowfall dates here...

        if isinstance(mx_24hr_sn_dt, str): # if no max snowfall value, then date values are set to nan, so check for this
        
            bdt = list(filter(None, (mx_24hr_sn_dt.split('TO')[0].split(' '))))[0]
            edt = list(filter(None, (mx_24hr_sn_dt.split('TO')[-1].split(' '))))[0]

            # if the dates are the same, then format for one day
            if bdt == edt:
                mx_24hr_sn_dt = _find_numeric_suffix(bdt.split('/')[-1])

                if isinstance(mx_24hr_sn, float) and mx_24hr_sn >= 10.0:
                    space3 = space*9
                    space4 = space*6
                    
                elif isinstance(mx_24hr_sn, float) and mx_24hr_sn <= 10.0:
                    space3 = space*15
                    space4 = space*10
                    
                elif mx_24hr_sn == 'T':
                    space3 = space*17
                    space4 = space*12
                    
            # if the dates are not the same, then format for multiple dates
            elif bdt != edt:
                bdt = _find_numeric_suffix(bdt.split('/')[-1])
                edt = _find_numeric_suffix(edt.split('/')[-1])
                mx_24hr_sn_dt = f'{bdt}-{edt}'

                if isinstance(mx_24hr_sn, float) and mx_24hr_sn >= 10.0:
                    space3 = space*9
                    space4 = space*4

                elif isinstance(mx_24hr_sn, float) and mx_24hr_sn <= 10.0:
                    space3 = space*10
                    space4 = space*5

                elif mx_24hr_sn == 'T':
                    space3 = space*11
                    space4 = space*6

            # format the max calendar day, and 24 hour snow fall values and dates (if applicable)
            mx_24hr_sn = f'{mx_24hr_sn}/{mx_24hr_sn_dt}'

        # lets format the max snow depth/dates
        if mx_sd > 0.0:
            dt = _find_numeric_suffix(mx_sd_dt.split('/')[-1])
            mx_sd = f'{int(mx_sd)}/{dt}'

        else:
            mx_sd = '0'
        
        f.write(f'{m}     {sn}{space1}{dfn}{space2}{mx_24hr_sn}{space3}{mx_24hr_sn}{space4}{mx_sd}\n')
        # end of for loop

    # -----------------------------------------------------------
    # now lets add the annual data max values at the bottom of the table

    # constant space values
    space1 = space*6 # space b/w the yearly snow and the yearly dfn
    space2 = space*7 # space b/w the yearly snow dfn and the calendar day max snow value/date

    # -------------------------------------------
    # get the annual snowfall total
    sn_filtered = [s for s in snow if s != 'T'] # filter out trace values
    
    # if the two lists are equal after filtering, then we filtered no T's
    if len(snow) == len(sn_filtered):
        yrly_sn = np.sum(sn_filtered)
    
    # if the two lists are not equal after filtering, then we filtered T's out...
    elif len(snow) != len(sn_filtered):
        if np.sum(sn_filtered) == 0.0:
            yrly_sn = 'T'
    
        elif np.sum(sn_filtered) > 0.0:
            yrly_sn = np.sum(sn_filtered)

    if isinstance(yrly_sn, float) and yrly_sn >= 10.0:
        space1 = space*5
        
    # -------------------------------------------
    # get the annual snow dfn
    yrly_sn_dfn = np.sum(snow_dfn)
    if yrly_sn_dfn > 10.0:
        yrly_sn_dfn = f'+{yrly_sn_dfn}'
        space2 = space*6

    elif yrly_sn_dfn == 0.0:
        yrly_sn_dfn = ' 0.0'

    # -------------------------------------------
    # get the max calendar day snow value and date
    

    # -----------------------------------------------------------
    f.write(table_sep)
    f.write(f'ANN.    {yrly_sn}{space1}{yrly_sn_dfn}{space2}0.4  JAN 31            0.4  JAN 31          0\n')
    



###########################################################################################################



# -----------------------------------------------------------
def _get_station_name(pil):
    if pil[3:] == 'LZK':
        stn_name = 'NORTH LITTLE ROCK'
    elif pil[3:] == 'LIT':
        stn_name = 'LITTLE ROCK'
    elif pil[3:] == 'HRO':
        stn_name = 'HARRISON'
    elif pil[3:] == 'PBF':
        stn_name = 'PINE BLUFF'

    return stn_name

# -----------------------------------------------------------

# will need to incorporate these into a function call...
pil = 'CLMLZK'
year = 2021

stn_name = _get_station_name(pil)

# a constant that will be used multiple times
table_sep = '--------------------------------------------------------------------------------------------\n'
space = ' '



    
# here we're actually writing the text file
with open(f'./test_supplemental_CLA{pil[3:]}_data.txt', 'w') as f:

    f.write(f'SUPPLEMENTAL ANNUAL CLIMATE DATA FOR {stn_name}\n')
    f.write('\n(DFN = DEPARTURE FROM NORMAL)\n')
    
    # first up, the annual temperature summary table
    _make_temp_table(f, year, stn_name)

    # next, write the annual precip summary table
    _make_precip_table(f, year, stn_name)

    # next, write the annual snow summary table
    _make_snow_table(f, year, stn_name)

    f.close()



    





In [464]:
# write our dictionary to a text file, so we can operate easily in pycharm without having to re acquire the data everytime...
with open('./2021_LZK_Annual_Summary.txt', 'w') as f:
    f.write(json.dumps(data_dict))

f.close()

In [467]:
with open('./2021_LZK_Annual_Summary.txt', 'r') as f:
    json_str = f.read()

f.close()

_new_data_dict = json.loads(json_str)

In [366]:
pcp = [data_dict[m]['monthly_rain_and_dfn'][0] for m in months]
pcp_dfn = [data_dict[m]['monthly_rain_and_dfn'][1] for m in months]

pcp_mx_clndr = [data_dict[m]['max_clndr_24hr_rain'][0] for m in months]
pcp_mx_clndr_dates = [data_dict[m]['max_clndr_24hr_rain'][1] for m in months]

precip_mx_storm_total = [data_dict[m]['max_clndr_24hr_rain'][2] for m in months]
precip_mx_storm_total_dates = [data_dict[m]['max_clndr_24hr_rain'][3] for m in months]

In [434]:
snow = [data_dict[m]['monthly_snow_sdepth_dfn'][0] for m in months if m in ['JAN', 'FEB', 'MAR', 'APR', 'OCT', 'NOV', 'DEC']]
snow_dfn = [data_dict[m]['monthly_snow_sdepth_dfn'][1] for m in months if m in ['JAN', 'FEB', 'MAR', 'APR', 'OCT', 'NOV', 'DEC']]

mx_sdepth = [data_dict[m]['monthly_snow_sdepth_dfn'][2] for m in months if m in ['JAN', 'FEB', 'MAR', 'APR', 'OCT', 'NOV', 'DEC']]
mx_sdepth_dt = [data_dict[m]['monthly_snow_sdepth_dfn'][3] for m in months if m in ['JAN', 'FEB', 'MAR', 'APR', 'OCT', 'NOV', 'DEC']]

mx_24hr_snow = [data_dict[m]['max_clndr_24hr_snow'][0] for m in months if m in ['JAN', 'FEB', 'MAR', 'APR', 'OCT', 'NOV', 'DEC']]
mx_24hr_snow_dt = [data_dict[m]['max_clndr_24hr_snow'][1] for m in months if m in ['JAN', 'FEB', 'MAR', 'APR', 'OCT', 'NOV', 'DEC']]

snow, snow_dfn, mx_sdepth, mx_sdepth_dt, mx_24hr_snow, mx_24hr_snow_dt

(['T', 18.7, 'T', 0.0, 0.0, 0.0, 0.0],
 [-1.7, 16.9, -0.4, 0.0, 0.0, -0.1, -0.6],
 [0.0, 14.0, 0.0, 0.0, 0.0, 0.0, 0.0],
 [nan, '02/18', nan, nan, nan, nan, nan],
 ['T', 10.9, 'T', 0.0, 0.0, 0.0, 0.0],
 ['01/15 TO 01/15', '02/14 TO 02/15', '03/31 TO 03/31', nan, nan, nan, nan])

In [450]:
list(filter('T', (snow)))

TypeError: 'str' object is not callable

In [448]:
snow_filtered = list(filter('T', snow))
snow_filtered

TypeError: 'str' object is not callable

In [453]:
sn_filtered = [s for s in snow if s != 'T'] # filter out trace values

# if the two lists are equal after filtering, then we filtered no T's
if len(snow) == len(sn_filtered):
    yrly_sn = np.sum(sn_filtered)

# if the two lists are not equal after filtering, then we filtered T's out...
elif len(snow) != len(sn_filtered):
    if np.sum(sn_filtered) == 0.0:
        yrly_sn = 'T'

    elif np.sum(sn_filtered) > 0.0:
        yrly_sn = np.sum(sn_filtered)

yrly_sn







    

18.7

In [360]:
import textwrap

#phrase = "bla bla bla ceci est une phrase beaucoup trop longue pour ce que je doit réellement dire, en fait je peux même dire qu'elle ne sert a rien, mais du coup je doit quand même la tester"
phrase = 'THE LOWEST YEARLY MINIMUM TEMPERATURE OF 18 DEGREES OCCURRED ON JAN 14TH, 31ST, AND MAR 19TH, WHICH WAS THE WARMEST LOW TEMPERATURE SINCE RECORD KEEPING BEGAN IN 1892 OR PUT ANOTHER WAY, THE 1ST HOTTEST MINIMUM TEMPERATURE OF ANY YEAR. THE NEW LOW OF 18 DEGREES SURPASSES THE PREVIOUS RECORD OF 16 DEGREES SET IN 1987.'


wrapper = textwrap.TextWrapper(width = 69) # this will be our wrapper function, limits characters to 69

#print(wrap_list)

with open("test__.txt", "w") as f:
    f.write(wrapper.fill(phrase)) # wrapper.fill automatically starts a new line when it wraps (adds '\n')


['bla bla bla ceci est une phrase beaucoup trop longue pour ce que je', "doit réellement dire, en fait je peux même dire qu'elle ne sert a", 'rien, mais du coup je doit quand même la tester']


In [ ]:
def get_precip_data(monthly_precip_data, monthly_snow_data, month):

    '''
    This function will parse the precip text info from the API url, and will append precip and snow data
    to the main working dictionary.

    monthly_precip_data : str, the parsed monthly precip data as a string
    monthly_snow_data : str, the parsed monthly snow data as a string
    month : int, the month as an integer

    returns: 
    None
    
    '''
    
    # Helper Sub-functions of the main function
    # --------------------------------------------------------
    def _check_record(param):
        if 'R' in param:
            return param.split('R')[0]
        else:
            return param

    # --------------------------------------------------------
    def _check_missing_snow(param, month):
        try:
            data_dict[month]['monthly_snow_sdepth_and_dfn'].append(float(param)) # monthly total
        
        except ValueError:
            data_dict[month]['monthly_snow_sdepth_and_dfn'].append(0.0)

    # --------------------------------------------------------
    
    # convert the integer month to the three letter month name
    month = calendar.month_name[month][:3].upper()


    
    # ---- Monthly Rain Data -----
    monthly_total_rain = _check_record(float(list(filter(None, (monthly_precip_data.split(' '))))[0])) # monthly total rain
    monthly_rain_dfn = _check_record(float(list(filter(None, (monthly_precip_data.split(' '))))[2]))
    
    # add the monthly precip total and dfn
    data_dict[month]['monthly_rain_and_dfn'].append(monthly_total_rain) # monthly total
    data_dict[month]['monthly_rain_and_dfn'].append(monthly_rain_dfn) # monthly dfn

    # now lets get the calendar max, 24 hr max values and dates for rain
    for idx, line in enumerate(monthly_precip_data.split('\nGREATEST\n 24 HR. TOTAL')[-1].splitlines()):
        if idx == 0:
            max_clndr_rain = float(list(filter(None, (line.split(' ')))[0]))
            max_clndr_rain_dates = " ".join(list(filter(None, (line.split(' ')))[1:]))

        elif idx == 1:
            max_stormtotal_rain = float(list(filter(None, (line.split(' ')))[-1]))
        
    #data_dict[month]['max_clndr_24hr_rain'].append(list(filter(None, (monthly_precip_data.split('\nGREATEST\n 24 HR. TOTAL')[-1].split(' '))))[0]) # 24 hr max (calendar day max)
    #data_dict[month]['max_clndr_24hr_rain'].append(list(filter(None, (monthly_precip_data.split('\nGREATEST\n 24 HR. TOTAL')[-1].split(' '))))[1:4]) # 24 hr max dates

    # 24 hour (calendar day max) rainfall data
    data_dict[month]['max_clndr_24hr_rain'].append(max_clndr_rain) # 24 hr max (calendar day max)
    data_dict[month]['max_clndr_24hr_rain'].append(max_clndr_rain_dates) # 24 hr max dates
    
    # storm total rain data
    #data_dict[month]['max_clndr_24hr_rain'].append(list(filter(None, (monthly_precip_data.split('\nGREATEST\n 24 HR. TOTAL')[-1].split(' '))))[7]) # storm total 
    #data_dict[month]['max_clndr_24hr_rain'].append(list(filter(None, (monthly_precip_data.split('\nGREATEST\n 24 HR. TOTAL')[-1].split(' '))))[1:4]) # 24 hr max dates (treat as same for now)
    
    data_dict[month]['max_clndr_24hr_rain'].append(max_stormtotal_rain) # max storm total 
    data_dict[month]['max_clndr_24hr_rain'].append(max_clndr_rain_dates) # max storm total dates (set equal to 24 hr max for now)

    
    
    # ---- Monthly Snow Data ----
    # add the monthly snow total and dfn
    monthly_total_snow = _check_record(list(filter(None, (monthly_snow_data.split(' '))))[0]) # monthly total snow
    monthly_grtst_sdepth = _check_record(list(filter(None, (monthly_snow_data.split('\nGREATEST\n SNOW DEPTH')[-1].splitlines()[0].split(' '))))[0]) # monthly greatest snow depth
    monthly_snow_dfn = _check_record(list(filter(None, (monthly_snow_data.split(' '))))[2]) # monthly total snow dfn

    # for the non winter months, monthly snow may just be set to 'MM' in the CLM product
    _check_missing_snow(monthly_total_snow, month) # this will add the monthly total snow
    _check_missing_snow(monthly_grtst_sdepth, month) # this will add the greatest monthly snow depth
    _check_missing_snow(monthly_snow_dfn, month) # this will add the monthly total snow dfn   


    

    

In [30]:
for idx, line in enumerate(monthly_precip_data.split('\nGREATEST\n 24 HR. TOTAL')[-1].splitlines()):
    #print(line.split(' '))
    # if idx == 0:
    #     print(" ".join(list(filter(None, (line.split(' '))))[1:]))
    if idx == 1:
        print(list(filter(None, (line.split(' '))))[-1])

1.81


In [36]:
monthly_snow_data.split('\nGREATEST\n SNOW DEPTH')[-1].splitlines()

monthly_sdepth = _check_record(list(filter(None, (monthly_snow_data.split('\nGREATEST\n SNOW DEPTH')[-1].splitlines()[0].split(' '))))[0])


['       14R  02/18                                           ',
 ' 24 HR TOTAL    10.9R  02/14 TO 02/15                0.3  02/20 TO 02/20']

In [41]:
list(filter(None, (monthly_snow_data.split('\nGREATEST\n SNOW DEPTH')[-1].splitlines()[0].split(' '))))[0]

'14R'

In [14]:
monthly_snow_data = text.split('SNOWFALL (INCHES)')[-1].split('\nTOTALS')[-1].split('\nDEGREE DAYS')[0]
monthly_snow_data

'          18.7R               1.8    16.9      0.3               \n LIQUID EQUIV   1.90               0.20    1.70     0.03               \nSINCE 7/1       18.8                4.0    14.8       MM               \n LIQUID 7/1     1.88               0.00    1.88       MM               \nSNOWDEPTH AVG.     2                                                   \nDAYS >= TRACE      6                1.2     4.8        3               \nDAYS >= 1.0        3                                   0               \nGREATEST\n SNOW DEPTH       14R  02/18                                           \n 24 HR TOTAL    10.9R  02/14 TO 02/15                0.3  02/20 TO 02/20\n'

In [14]:
float(list(filter(None, (monthly_precip_data.split(' '))))[0]), float(list(filter(None, (monthly_precip_data.split(' '))))[2])

(7.5, 3.96)

In [188]:
'''
Here we were working with the xmacis API to obtain data, we have working code to get annual avg data and departure from normal,
now we need some logic to compare a yearly value 

'''



def get_annual_xmacis_data(station):

    # a dictionary that contains the json syntax to the xmacis api, that will return yearly avg values for the por, and the departures from normal
    json_dict = {
        'lit' : '{"sid":"LITthr","sdate":"por","edate":"por","output":"json","elems":[{"name":"maxt","interval":"yly","duration":1,"reduce":"mean","maxmissing":"7","prec":1},{"name":"maxt","interval":"yly","duration":1,"reduce":"mean","maxmissing":"7","normal":"departure","prec":1},{"name":"mint","interval":"yly","duration":1,"reduce":"mean","maxmissing":"7","prec":1},{"name":"mint","interval":"yly","duration":1,"reduce":"mean","maxmissing":"7","normal":"departure","prec":1},{"name":"avgt","interval":"yly","duration":1,"reduce":"mean","maxmissing":"7","prec":1},{"name":"avgt","interval":"yly","duration":1,"reduce":"mean","maxmissing":"7","normal":"departure","prec":1},{"name":"pcpn","interval":"yly","duration":1,"reduce":"sum","maxmissing":"7","prec":1},{"name":"pcpn","interval":"yly","duration":1,"reduce":"sum","maxmissing":"7","normal":"departure","prec":1},{"name":"snow","interval":"yly","duration":1,"reduce":"sum","maxmissing":"7","prec":1},{"name":"snow","interval":"yly","duration":1,"reduce":"sum","normal":"departure","maxmissing":"7","prec":1}],"output":"json"}',
        'lzk' : '{"sid":"LZKthr","sdate":"por","edate":"por","output":"json","elems":[{"name":"maxt","interval":"yly","duration":1,"reduce":"mean","maxmissing":"7","prec":1},{"name":"maxt","interval":"yly","duration":1,"reduce":"mean","maxmissing":"7","normal":"departure","prec":1},{"name":"mint","interval":"yly","duration":1,"reduce":"mean","maxmissing":"7","prec":1},{"name":"mint","interval":"yly","duration":1,"reduce":"mean","maxmissing":"7","normal":"departure","prec":1},{"name":"avgt","interval":"yly","duration":1,"reduce":"mean","maxmissing":"7","prec":1},{"name":"avgt","interval":"yly","duration":1,"reduce":"mean","maxmissing":"7","normal":"departure","prec":1},{"name":"pcpn","interval":"yly","duration":1,"reduce":"sum","maxmissing":"7","prec":1},{"name":"pcpn","interval":"yly","duration":1,"reduce":"sum","maxmissing":"7","normal":"departure","prec":1},{"name":"snow","interval":"yly","duration":1,"reduce":"sum","maxmissing":"7","prec":1},{"name":"snow","interval":"yly","duration":1,"reduce":"sum","normal":"departure","maxmissing":"7","prec":1}],"output":"json"}',
        'hro' : '{"sid":"HROthr","sdate":"por","edate":"por","output":"json","elems":[{"name":"maxt","interval":"yly","duration":1,"reduce":"mean","maxmissing":"7","prec":0},{"name":"maxt","interval":"yly","duration":1,"reduce":"mean","maxmissing":"7","normal":"departure","prec":0},{"name":"mint","interval":"yly","duration":1,"reduce":"mean","maxmissing":"7","prec":1},{"name":"mint","interval":"yly","duration":1,"reduce":"mean","maxmissing":"7","normal":"departure","prec":1},{"name":"avgt","interval":"yly","duration":1,"reduce":"mean","maxmissing":"7","prec":1},{"name":"avgt","interval":"yly","duration":1,"reduce":"mean","maxmissing":"7","normal":"departure","prec":1},{"name":"pcpn","interval":"yly","duration":1,"reduce":"sum","maxmissing":"7","prec":1},{"name":"pcpn","interval":"yly","duration":1,"reduce":"sum","maxmissing":"7","normal":"departure","prec":1},{"name":"snow","interval":"yly","duration":1,"reduce":"sum","maxmissing":"7","prec":1},{"name":"snow","interval":"yly","duration":1,"reduce":"sum","normal":"departure","maxmissing":"7","prec":1}],"output":"json"}',
        'pbf' : '{"sid":"PBFthr","sdate":"por","edate":"por","output":"json","elems":[{"name":"maxt","interval":"yly","duration":1,"reduce":"mean","maxmissing":"7","prec":1},{"name":"maxt","interval":"yly","duration":1,"reduce":"mean","maxmissing":"7","normal":"departure","prec":1},{"name":"mint","interval":"yly","duration":1,"reduce":"mean","maxmissing":"7","prec":1},{"name":"mint","interval":"yly","duration":1,"reduce":"mean","maxmissing":"7","normal":"departure","prec":1},{"name":"avgt","interval":"yly","duration":1,"reduce":"mean","maxmissing":"7","prec":1},{"name":"avgt","interval":"yly","duration":1,"reduce":"mean","maxmissing":"7","normal":"departure","prec":1},{"name":"pcpn","interval":"yly","duration":1,"reduce":"sum","maxmissing":"7","prec":1},{"name":"pcpn","interval":"yly","duration":1,"reduce":"sum","maxmissing":"7","normal":"departure","prec":1},{"name":"snow","interval":"yly","duration":1,"reduce":"sum","maxmissing":"7","prec":1},{"name":"snow","interval":"yly","duration":1,"reduce":"sum","normal":"departure","maxmissing":"7","prec":1}],"output":"json"}'
    }

    json_text = json_dict[station]
    
    # the xmacis api server for station data
    url = f"https://data.rcc-acis.org/StnData?params={json_text}"
    
    # A GET request to the API
    r = requests.get(url)
    return r.json()

data = get_annual_xmacis_data('hro')

# --------------------------------------------------------------------

# this will take the json dictionary from the api, and create lists of lists for each annual avg parameter
maxt = [item[:3] for item in data['data']]
mint = []
avgt = []
pcpn = []
snow = []

for item in data['data']:

    mint.append([item[0], item[3], item[4]])
    avgt.append([item[0], item[5], item[6]])
    pcpn.append([item[0], item[7], item[8]])
    snow.append([item[0], item[9], item[10]])


In [190]:
def sort_annual_temps(temps, mintemps = False):

    '''
    This function will sort a list of annual avg temp values, and delete missing years

    temps : list of lists, a list of lists of annual temps for a station, including the year, avg value, and departure from normal
    mintemps : boolean, default set to False, if True, then returned order will be reversed to represent the coldest temps at the top

    returns:
    param_filtered : a sorted and filtered list of annual avg temps

    '''
    
    index = [1, 2]

    # for precip parameters this converts the values that are not missing or trace to floats
    param_filtered =[
            [np.float64(val) if i in index and val not in ['M'] else val for i, val in enumerate(subl)] 
            for subl in temps
            ]
    
    # now lets convert missings to nan, nans arrise from the 7 missing days in a year threshold
    param_filtered = [
            [np.nan if i in index and val == 'M' else val for i, val in enumerate(subl)] 
            for subl in param_filtered
            ]

    # now convert the years to integers
    index = [0]
    
    param_filtered =[
        [int(val) if i in index and val not in ['M'] else val for i, val in enumerate(subl)] 
        for subl in param_filtered
        ]
    
    # this will remove the nan values for sorting the data
    param_filtered = [item for item in param_filtered if item[1] is not np.nan]
    
    # this will create a sorted list, with the largest values first
    if mintemps:
        return sorted(param_filtered, key=itemgetter(1), reverse = False)
    else:
        return sorted(param_filtered, key=itemgetter(1), reverse = True)


maxt_filtered = sort_annual_temps(maxt)
mint_filtered = sort_annual_temps(mint, mintemps = True)
avgt_filtered = sort_annual_temps(avgt)




In [212]:
searchyear = 2023

for idx, lst in enumerate(maxt_filtered):
    #print(idx+1, lst)

    if lst[0] == searchyear:
        rank = idx+1
        break

percentile = (rank/len(maxt_filtered))*100

filtered_data = maxt_filtered[:idx]
# calculate residules over the filtered list to find the min residual and thus, the next closest year
res = []
for sublst in filtered_data:
    res.append(abs(searchyear - sublst[0]))

next_closest_year = filtered_data[res.index(min(res))][0]

if percentile < 50:
    print(f'The yearly average high temperature of {searchyear} was the {rank}th warmest year, and the warmest year since {next_closest_year}.')





found our year at rank 15
The yearly average high temperature of 2023 was the 15th warmest year, and the warmest year since 2017.


In [206]:
filtered_data

res = []
for sublst in filtered_data:
    res.append(abs(searchyear - sublst[0]))

print(res.index(min(res)))








13


In [162]:
def sort_annual_precip_value(param):

    '''
    This function will sort a list of annual total snowfall or precip values, converting Trace values to 0.005, and deleting missing years

    param : list of lists, a list of lists of snowfall or precip for a station, including the year, total value, and departure from normal

    returns:
    param_filtered : a sorted and filtered list of annual param totals

    '''
    
    index = [1, 2]

    # for precip parameters this converts the values that are not missing or trace to floats
    param_filtered =[
            [np.float64(val) if i in index and val not in ['M', 'T'] else val for i, val in enumerate(subl)] 
            for subl in param
            ]
    
    # now lets convert missings to nan, nans arrise from the 7 missing days in a year threshold
    param_filtered = [
            [np.nan if i in index and val == 'M' else val for i, val in enumerate(subl)] 
            for subl in param_filtered
            ]
    
    # lastly lets convert trace to 0.005
    param_filtered = [
            [np.float64(0.005) if i in index and val == 'T' else val for i, val in enumerate(subl)] 
            for subl in param_filtered
            ]

    # now convert the years to integers
    index = [0]
    
    param_filtered =[
        [int(val) if i in index and val not in ['M'] else val for i, val in enumerate(subl)] 
        for subl in param_filtered
        ]
    
    # this will remove the nan values for sorting the data
    param_filtered = [item for item in param_filtered if item[1] is not np.nan]
    
    # this will create a sorted list, with the largest values first
    return sorted(param_filtered, key=itemgetter(1), reverse = True)


filtered_snow = sort_annual_precip_value(snow)

In [81]:
url = 'https://mesonet.agron.iastate.edu/api/1/nwstext/202302051153-KLZK-CXUS54-CLMLZK'

html = urlopen(url).read()
soup = BeautifulSoup(html, features="html.parser")

# kill all script and style elements
for script in soup(["script", "style"]):
    script.extract()    # rip it out

# get text
text = soup.get_text()


text

"254 \nCXUS54 KLZK 081205\nCLMLZK\n\nCLIMATE REPORT...UPDATED WITH SUPPLEMENTAL DATA \nNATIONAL WEATHER SERVICE LITTLE ROCK AR\n553 AM CST SUN FEB 05 2023\n\n...................................\n\n...THE NORTH LITTLE ROCK CLIMATE SUMMARY FOR THE MONTH OF JANUARY 2023...\n\nCLIMATE NORMAL PERIOD 1991 TO 2020\nCLIMATE RECORD PERIOD 1975 TO 2023\n\nWEATHER         OBSERVED          NORMAL  DEPART   LAST YEAR'S         \n                VALUE   DATE(S)   VALUE   FROM     VALUE DATE(S)       \n                                          NORMAL                       \n................................................................\nTEMPERATURE (F)\nRECORD\n HIGH             81   01/30/2002                                      \n LOW              -6   01/20/1985                                      \nHIGHEST           72   01/11         49      23       68  01/01        \nLOWEST            24   01/31         31      -7       13  01/21        \n                       01/14                      

In [89]:
avg_high = text.split('AVG. MAXIMUM')[-1].split('AVG. MINIMUM')[0] # with this, the first value would be the monthly avg, the 3rd value would be the DFN
avg_low = text.split('AVG. MINIMUM')[-1].split('MEAN')[0] 

mon_avg_temp = text.split('MEAN')[-1].split('DAYS MAX >= 90')[0]
mon_avg_temp

'            47.0               40.6     6.4     39.4               \n'

In [57]:
url = 'https://mesonet.agron.iastate.edu/api/1/nws/afos/list.json?cccc=KLZK&pil=CLMLIT&date=2023-02-05'

# A GET request to the API
response = requests.get(url)

# turn the request into a json, and subsequently a dictionary
response = response.json()

# convert the json to dictionary
#response = f'[{response}]'
#response = json.loads(response)[0]
print(response, type(response))

{'schema': {'fields': [{'name': 'index', 'type': 'integer'}, {'name': 'entered', 'type': 'string'}, {'name': 'pil', 'type': 'string'}, {'name': 'product_id', 'type': 'string'}, {'name': 'cccc', 'type': 'string'}, {'name': 'count', 'type': 'integer'}, {'name': 'link', 'type': 'string'}, {'name': 'text_link', 'type': 'string'}], 'primaryKey': ['index'], 'pandas_version': '1.4.0'}, 'data': [{'index': 0, 'entered': '2023-02-05T11:53:00Z', 'pil': 'CLMLIT', 'product_id': '202302051153-KLZK-CXUS54-CLMLIT', 'cccc': 'KLZK', 'count': 2, 'link': 'https://mesonet.agron.iastate.edu/p.php?pid=202302051153-KLZK-CXUS54-CLMLIT', 'text_link': 'https://mesonet.agron.iastate.edu/api/1/nwstext/202302051153-KLZK-CXUS54-CLMLIT'}]} <class 'dict'>


In [59]:
response['data'][0]

{'index': 0,
 'entered': '2023-02-05T11:53:00Z',
 'pil': 'CLMLIT',
 'product_id': '202302051153-KLZK-CXUS54-CLMLIT',
 'cccc': 'KLZK',
 'count': 2,
 'link': 'https://mesonet.agron.iastate.edu/p.php?pid=202302051153-KLZK-CXUS54-CLMLIT',
 'text_link': 'https://mesonet.agron.iastate.edu/api/1/nwstext/202302051153-KLZK-CXUS54-CLMLIT'}

In [29]:
url = 'https://mesonet.agron.iastate.edu/api/1/nws/afos/list.json?cccc=KLZK&pil=CLMLIT&date=2023-02-05'
#url = f'https://mesonet.agron.iastate.edu/api/1/nws/afos/list.json?cccc=KLZK&pil={station}&date={y}-{m:02}-{d:02}'


html = urlopen(url).read()
soup = BeautifulSoup(html, features="html.parser")

# kill all script and style elements
for script in soup(["script", "style"]):
    script.extract()    # rip it out

text = soup.get_text()

#text.split('Skip to cccc:')[-1]

text = f'[{text}]'
json_item = json.loads(text)[0]

# if not json_item['data']:
#     print('no product issued')

# elif json_item['data']:
#     print(json_item['data']['text_link'])

In [35]:
json_item['data'][0]['text_link']

'https://mesonet.agron.iastate.edu/api/1/nwstext/202302051153-KLZK-CXUS54-CLMLIT'

In [24]:
text_ = f'[{text}]'
#print(text_)
json1_data = json.loads(text_)[0]
json1_data['data']

[]

In [26]:
if not json1_data['data']:
    print('no product issued')

no product issued


In [7]:
text = text.split('Skip to cccc:')[-1].split('College of Ag\n')[0]


# now lets iterate through the four PILs
for station in ['LIT', 'LZK', 'HRO', 'PBF']:
    if f'CLM{station}' in text:

        product_time = text.split(f'CLM{station}')[-1].split('@')[-1].split('\n')[0]
        print(product_time)


13:56
13:56
13:56
13:56


In [13]:


text.split('CLMLIT')[-1].split('x@')[-1].split('\n')[0]

'11:54CLMLZK @11:54CLMPBF @11:54'

In [151]:
# use the api page to parse the actual text

#api_url = f'https://mesonet.agron.iastate.edu/api/1/nwstext/202302051153-KLZK-CXUS54-CLMLZK'
api_url = f'https://mesonet.agron.iastate.edu/api/1/nwstext/202303041555-KLZK-CXUS54-CLMLIT'

html = urlopen(api_url).read()
soup = BeautifulSoup(html, features="html.parser")

# kill all script and style elements
for script in soup(["script", "style"]):
    script.extract()    # rip it out

# get text
text = soup.get_text()

In [221]:
def text_parser(timestamp, pil):

    y = timestamp.year
    m = timestamp.month
    d = timestamp.day
    hh = timestamp.hour
    mm = timestamp.minute
    
    api_url = f'https://mesonet.agron.iastate.edu/api/1/nwstext/{y}{m:02}{d:02}{hh:02}{mm:02}-KLZK-CXUS54-{pil}'
    print(api_url)
    
    html = urlopen(api_url).read()
    soup = BeautifulSoup(html, features="html.parser")
    
    # kill all script and style elements
    for script in soup(["script", "style"]):
        script.extract()    # rip it out
    
    # get text
    text = soup.get_text()

    # parse out the highest and lowest temp data for the month from the text
    high_temp_data = text.split('HIGHEST         ')[-1].split('LOWEST')[0].splitlines()   
    low_temp_data = text.split('LOWEST           ')[-1].split('AVG. MAXIMUM')[0]  

    get_maxmin_data(high_temp_data, low_temp_data, month)

text_parser(timestamp, 'CLMHRO')
    

https://mesonet.agron.iastate.edu/api/1/nwstext/202306060355-KLZK-CXUS54-CLMHRO
104 
CXUS54 KLZK 060357
CLMHRO

CLIMATE REPORT...UPDATED WITH SUPPLEMENTAL DATA 
NATIONAL WEATHER SERVICE LITTLE ROCK AR 
1055 PM CDT MON JUN 05 2023

...................................

...THE HARRISON CLIMATE SUMMARY FOR THE MONTH OF MAY 2023...

CLIMATE NORMAL PERIOD 1991 TO 2020
CLIMATE RECORD PERIOD 1891 TO 2023

WEATHER         OBSERVED          NORMAL  DEPART   LAST YEAR'S         
                VALUE   DATE(S)   VALUE   FROM     VALUE DATE(S)       
                                          NORMAL                       
................................................................
TEMPERATURE (F)
RECORD
 HIGH             99   05/29/1926                                      
                       05/28/1926                                      
 LOW              26   05/02/1911                                      
                       05/01/1903                                      
HIGHEST

In [217]:
y = 2023
m = 6
d = 5
hh = 22
mm = 55



In [218]:
def parse_timestamp(y, m, d, hh, mm):

    timestamp = datetime(y, m, d, hh, mm)
    timestamp = timestamp.astimezone()
    #return timestamp.astimezone(timezone.utc)


timestamp = parse_timestamp(y, m, d, hh, mm)

In [152]:
# text_r = text.splitlines()

# for line in text_r:
#     print(line)

234 
CXUS54 KLZK 041555
CLMLIT

CLIMATE REPORT 
NATIONAL WEATHER SERVICE LITTLE ROCK AR
955 AM CST SAT MAR 04 2023

...................................

...THE LITTLE ROCK ADAMS FIELD CLIMATE SUMMARY FOR THE MONTH OF FEBRUARY 2023...

CLIMATE NORMAL PERIOD 1991 TO 2020
CLIMATE RECORD PERIOD 1874 TO 2023

WEATHER         OBSERVED          NORMAL  DEPART   LAST YEAR'S         
                VALUE   DATE(S)   VALUE   FROM     VALUE DATE(S)       
                                          NORMAL                       
................................................................
TEMPERATURE (F)
RECORD
 HIGH             87   02/25/1918                                      
 LOW             -12   02/12/1899                                      
HIGHEST           81   02/28         56      25       72  02/11        
                                                          02/17        
LOWEST            27   02/18         35      -8       23  02/05        
                       02/04  

In [153]:
text_r

['234 ',
 'CXUS54 KLZK 041555',
 'CLMLIT',
 '',
 'CLIMATE REPORT ',
 'NATIONAL WEATHER SERVICE LITTLE ROCK AR',
 '955 AM CST SAT MAR 04 2023',
 '',
 '...................................',
 '',
 '...THE LITTLE ROCK ADAMS FIELD CLIMATE SUMMARY FOR THE MONTH OF FEBRUARY 2023...',
 '',
 'CLIMATE NORMAL PERIOD 1991 TO 2020',
 'CLIMATE RECORD PERIOD 1874 TO 2023',
 '',
 "WEATHER         OBSERVED          NORMAL  DEPART   LAST YEAR'S         ",
 '                VALUE   DATE(S)   VALUE   FROM     VALUE DATE(S)       ',
 '                                          NORMAL                       ',
 '................................................................',
 'TEMPERATURE (F)',
 'RECORD',
 ' HIGH             87   02/25/1918                                      ',
 ' LOW             -12   02/12/1899                                      ',
 'HIGHEST           81   02/28         56      25       72  02/11        ',
 '                                                          02/17        ',
 '

In [ ]:
def get_maxmin_data(high_data, low_data):

    '''
    Worker function to extract high and low data/dates from the CLM products.
    
    '''
    
    # filter out the '' values inside the lists
    high_data = list(filter(None, high_data))
    low_data = list(filter(None, low_data))

    print('high data')
    print(high_data)
    print('low data')
    print(low_data)
    print('\n')

    # now lets get some data
    data_dict['highs'].append(high_data[0])
    data_dict['lows'].append(low_data[0])

    data_dict['high_dates'].append([high_data[1]])
    data_dict['low_dates'].append([low_data[1]])

In [231]:
# establish our main working dictionary
data_dict = {}

for s in ['LIT', 'LZK', 'PBF', 'HRO']:
    _data_dict = {
        f'{s}' : {}
    }
    data_dict.update(_data_dict)

# now add the months
for key in data_dict:
    for i in range(1,13):
        _data_dict = {
             f'{calendar.month_name[i][:3].upper()}' : {# generate the abbreviated month names
            'highs': [],
            'high_dates': [], # these should still contain 12 items, but will be lists inside lists in case of multiple dates
            'lows': [],
            'low_dates': []  # these should still contain 12 items, but will be lists inside lists in case of multiple dates                                     }
                }
            }
        data_dict[key].update(_data_dict)


data_dict

{'LIT': {'JAN': {'highs': [], 'high_dates': [], 'lows': [], 'low_dates': []},
  'FEB': {'highs': [], 'high_dates': [], 'lows': [], 'low_dates': []},
  'MAR': {'highs': [], 'high_dates': [], 'lows': [], 'low_dates': []},
  'APR': {'highs': [], 'high_dates': [], 'lows': [], 'low_dates': []},
  'MAY': {'highs': [], 'high_dates': [], 'lows': [], 'low_dates': []},
  'JUN': {'highs': [], 'high_dates': [], 'lows': [], 'low_dates': []},
  'JUL': {'highs': [], 'high_dates': [], 'lows': [], 'low_dates': []},
  'AUG': {'highs': [], 'high_dates': [], 'lows': [], 'low_dates': []},
  'SEP': {'highs': [], 'high_dates': [], 'lows': [], 'low_dates': []},
  'OCT': {'highs': [], 'high_dates': [], 'lows': [], 'low_dates': []},
  'NOV': {'highs': [], 'high_dates': [], 'lows': [], 'low_dates': []},
  'DEC': {'highs': [], 'high_dates': [], 'lows': [], 'low_dates': []}},
 'LZK': {'JAN': {'highs': [], 'high_dates': [], 'lows': [], 'low_dates': []},
  'FEB': {'highs': [], 'high_dates': [], 'lows': [], 'low_date

In [226]:
test_dict = {
    'LIT' : {
        'JAN' : {
            'highs' : [],
            'lows' : []
        }
    }
}
test_dict['LIT']['JAN']

{'highs': [], 'lows': []}

In [224]:
test_dict['LIT']['JAN'] = []
test_dict

KeyError: 'LIT'

In [183]:
# parse out the highest and lowest temp data for the month from the text
high_temp_data = text.split('HIGHEST         ')[-1].split('LOWEST')[0].splitlines()   
low_temp_data = text.split('LOWEST           ')[-1].split('AVG. MAXIMUM')[0]      

print(low_temp_data)
# multiple lines would indicate multiple dates of occurrence of the lowest temp
# if len(low_temp_data) > 1:

#     # first get temp and first date of occurrence
#     print(low_temp_data[0].split(' '))
    



    

 27   02/18         35      -8       23  02/05        
                       02/04                                           
                                                          02/06        
                                                          02/13        



In [177]:
_low_temp_data = list(filter(None, low_temp_data.split(' ')))

print(_low_temp_data)

['27', '02/18', '35', '-8', '23', '02/05', '\n', '02/04', '\n', '02/06', '\n', '02/13', '\n']


In [186]:

def get_maxmin_data(high_data, low_data, month):

    '''
    Worker function to extract high and low data/dates from the CLM products.
    
    '''

    # first filter the line of temp data
    filtered_low_temp_data = list(filter(None, low_temp_data.split(' '))) # low temp data
    filtered_high_temp_data = list(filter(None, high_temp_data.split(' '))) # high temp data
    
    # lets add the high and low values to the dictionary
    data_dict[month]['lows'].append(filtered_low_temp_data[0])
    data_dict[month]['highs'].append(filtered_high_temp_data[0])

    
    ## LOW TEMP DATA DATES ##
    #---------------------------------------------------------------
    # check how many lines exist in the list
    # if only one line, then just one date to grab
    if len(low_temp_data.splitlines()) == 1:
    
        data_dict[month]['low_dates'].append([filtered_low_temp_data[1]])
    
    # if there are multiple rows of low temp data, then multiple dates to grab
    elif len(low_temp_data.splitlines()) > 1:
        dates = []
        
        # grab the first date in the line
        dates.append(filtered_low_temp_data[1])
    
        # now iterate and find the rest of the dates
        for line in low_temp_data.splitlines()[1:]:
            if count_spaces(line):
                dates.append(list(filter(None, (line.split(' '))))[0])
    
        data_dict[month]['low_dates'].append(dates)
    
    
    ## HIGH TEMP DATA DATES ##
    #---------------------------------------------------------------
    # check how many lines exist in the list
    # if only one line, then just one date to grab
    if len(high_temp_data.splitlines()) == 1:
    
        data_dict[month]['high_dates'].append([filtered_high_temp_data[1]])
    
    # if there are multiple rows of high temp data, then multiple dates to grab
    elif len(high_temp_data.splitlines()) > 1:
        dates = []
        
        # grab the first date in the line
        dates.append(filtered_high_temp_data[1])
    
        # now iterate and find the rest of the dates
        for line in high_temp_data.splitlines()[1:]:
            if count_spaces(line):
                dates.append(list(filter(None, (line.split(' '))))[0])
    
        data_dict[month]['high_dates'].append(dates)

    
#data_dict

In [181]:
data_dict

{'JAN': {'highs': [], 'high_dates': [], 'lows': [], 'low_dates': []},
 'FEB': {'highs': [],
  'high_dates': [],
  'lows': [' 27   02/18         35      -8       23  02/05        ',
   ' 27   02/18         35      -8       23  02/05        ',
   '',
   '',
   ''],
  'low_dates': []},
 'MAR': {'highs': [], 'high_dates': [], 'lows': [], 'low_dates': []},
 'APR': {'highs': [], 'high_dates': [], 'lows': [], 'low_dates': []},
 'MAY': {'highs': [], 'high_dates': [], 'lows': [], 'low_dates': []},
 'JUN': {'highs': [], 'high_dates': [], 'lows': [], 'low_dates': []},
 'JUL': {'highs': [], 'high_dates': [], 'lows': [], 'low_dates': []},
 'AUG': {'highs': [], 'high_dates': [], 'lows': [], 'low_dates': []},
 'SEP': {'highs': [], 'high_dates': [], 'lows': [], 'low_dates': []},
 'OCT': {'highs': [], 'high_dates': [], 'lows': [], 'low_dates': []},
 'NOV': {'highs': [], 'high_dates': [], 'lows': [], 'low_dates': []},
 'DEC': {'highs': [], 'high_dates': [], 'lows': [], 'low_dates': []}}

In [168]:
test_string = '                       02/04                                           '

test_string.count(' ')

66

In [160]:
split_date = list(filter(None, (test_string.split(' '))))[0]

'02/04'

In [169]:
def count_spaces(line):

    # this should be the date in the line
    split_date = list(filter(None, (line.split(' '))))[0]

    # this is the number of spaces for it to be a date occurred that month
    if line.split(split_date)[0].count(' ') == 23:
        return True

    else:
        return False


# if count_spaces(test_string):
#     print('true')
# else:
#     print('false')

true


In [27]:
# here we're just importing datafiles, but first lets import from the api url

In [34]:
# this is the api url, it just displays the data in raw text

url = 'https://mesonet.agron.iastate.edu/api/1/nwstext/202401151935-KLZK-CXUS54-CLMLIT'


html = urlopen(url).read()
soup = BeautifulSoup(html, features="html.parser")

# kill all script and style elements
for script in soup(["script", "style"]):
    script.extract()    # rip it out

#print(text)

# get text
text = soup.get_text()

print(text)

447 
CXUS54 KLZK 151933
CLMLIT

CLIMATE REPORT...UPDATED WITH SUPPLEMENTAL DATA 
NATIONAL WEATHER SERVICE LITTLE ROCK AR
0135 PM CST MON JAN 15 2024

...................................

...THE LITTLE ROCK ADAMS FIELD CLIMATE SUMMARY FOR THE MONTH OF DECEMBER 2023...

CLIMATE NORMAL PERIOD 1991 TO 2020
CLIMATE RECORD PERIOD 1874 TO 2023

WEATHER         OBSERVED          NORMAL  DEPART   LAST YEAR'S         
                VALUE   DATE(S)   VALUE   FROM     VALUE DATE(S)       
                                          NORMAL                       
................................................................
TEMPERATURE (F)
RECORD
 HIGH             81   12/26/2015                                      
 LOW              -1   12/23/1989                                      
                       12/24/1963                                      
HIGHEST           69   12/01         53      16       74  12/29        
LOWEST            29   12/19         34      -5        2  12/23     

In [35]:
text

"447 \nCXUS54 KLZK 151933\nCLMLIT\n\nCLIMATE REPORT...UPDATED WITH SUPPLEMENTAL DATA \nNATIONAL WEATHER SERVICE LITTLE ROCK AR\n0135 PM CST MON JAN 15 2024\n\n...................................\n\n...THE LITTLE ROCK ADAMS FIELD CLIMATE SUMMARY FOR THE MONTH OF DECEMBER 2023...\n\nCLIMATE NORMAL PERIOD 1991 TO 2020\nCLIMATE RECORD PERIOD 1874 TO 2023\n\nWEATHER         OBSERVED          NORMAL  DEPART   LAST YEAR'S         \n                VALUE   DATE(S)   VALUE   FROM     VALUE DATE(S)       \n                                          NORMAL                       \n................................................................\nTEMPERATURE (F)\nRECORD\n HIGH             81   12/26/2015                                      \n LOW              -1   12/23/1989                                      \n                       12/24/1963                                      \nHIGHEST           69   12/01         53      16       74  12/29        \nLOWEST            29   12/19         34   

In [38]:
split_text = text.split('\n')

In [49]:
temp_data = text.split('\n................................................................\nTEMPERATURE (F)\n')[-1].split('\nPRECIPITATION (INCHES)\n')[0]
temp_data

'RECORD\n HIGH             81   12/26/2015                                      \n LOW              -1   12/23/1989                                      \n                       12/24/1963                                      \nHIGHEST           69   12/01         53      16       74  12/29        \nLOWEST            29   12/19         34      -5        2  12/23        \nAVG. MAXIMUM    59.8               52.6     7.2     53.9               \nAVG. MINIMUM    37.9               33.3     4.6     36.5               \nMEAN            48.8               43.0     5.8     45.2               \nDAYS MAX >= 90     0                0.0     0.0        0               \nDAYS MAX <= 32     0                0.9    -0.9        1               \nDAYS MIN <= 32     7               13.4    -6.4       12               \nDAYS MIN <= 0      0                0.0     0.0        0               \n'

In [76]:
temp_data = text.split('\n................................................................\nTEMPERATURE (F)\n')[-1].split('\nPRECIPITATION (INCHES)\n')[0]
high = temp_data.split('\nHIGHEST')[-1].split(' ')
low = temp_data.split('\nLOWEST')[-1].split(' ')


['', '', '', '', '', '', '', '', '', '', '', '', '29', '', '', '12/19', '', '', '', '', '', '', '', '', '34', '', '', '', '', '', '-5', '', '', '', '', '', '', '', '2', '', '12/23', '', '', '', '', '', '', '', '\nAVG.', 'MAXIMUM', '', '', '', '59.8', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '52.6', '', '', '', '', '7.2', '', '', '', '', '53.9', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '\nAVG.', 'MINIMUM', '', '', '', '37.9', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '33.3', '', '', '', '', '4.6', '', '', '', '', '36.5', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '\nMEAN', '', '', '', '', '', '', '', '', '', '', '', '48.8', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '43.0', '', '', '', '', '5.8', '', '', '', '', '45.2', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '\nDAYS', 'MAX', '>=', '90', '', '', '', '', '0', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '0.0', '', '', '', '', '0.0', '', '

In [93]:
data_dict = {

    'mon'  : [calendar.month_name[i][:3].upper() for i in range(1,13)], # generate the abbreviated month names
    'highs': [],
    'high_dates': [], # these should still contain 12 items, but will be lists inside lists in case of multiple dates
    'lows': [],
    'low_dates': []  # these should still contain 12 items, but will be lists inside lists in case of multiple dates
}

In [117]:
def get_maxmin_data(high_data, low_data):

    '''
    Worker function to extract high and low data/dates from the CLM products.
    
    '''
    
    # filter out the '' values inside the lists
    high_data = list(filter(None, high_data))
    low_data = list(filter(None, low_data))

    print('high data')
    print(high_data)
    print('low data')
    print(low_data)
    print('\n')

    # now lets get some data
    data_dict['highs'].append(high_data[0])
    data_dict['lows'].append(low_data[0])

    data_dict['high_dates'].append([high_data[1]])
    data_dict['low_dates'].append([low_data[1]])


# establish our main working dictionary
data_dict = {

    'mon'  : [calendar.month_name[i][:3].upper() for i in range(1,13)], # generate the abbreviated month names
    'highs': [],
    'high_dates': [], # these should still contain 12 items, but will be lists inside lists in case of multiple dates
    'lows': [],
    'low_dates': []  # these should still contain 12 items, but will be lists inside lists in case of multiple dates
}

# working directory, this wont exist in the final product (will poll from the IEM API)
directory = 'D:/Climate/'

for file in os.listdir(directory):
    #print(file)
    f = os.path.join(directory, file)
    f = open(f,'r')
    
    text = f.readlines() # this extracts all the text data from each file

    text = ''.join(text)
    #print(text)

    # here we are splitting the data to get to the monthly temp information
    temp_data = text.split('\n................................................................\nTEMPERATURE (F)\n')[-1].split('\nPRECIPITATION (INCHES)\n')[0]
    
    # splits to parse out the high and low temp data for the month
    #print('high data')
    high_data = temp_data.split('\nHIGHEST')[-1].split('\nLOWEST')[0].split(' ')
    #print('low data\n')
    low_data = temp_data.split('\nLOWEST')[-1].split('\nAVG')[0].split(' ')
    
        
        
    get_maxmin_data(high_data, low_data)


data_dict

high data
['77', '01/11', '50', '27', '72', '01/13']
low data
['25', '01/14', '31', '-6', '17', '01/22']


high data
['81', '02/28', '56', '25', '72', '02/11', '\n', '02/17']
low data
['27', '02/18', '35', '-8', '23', '02/05', '\n', '02/04', '\n', '02/06', '\n', '02/13']


high data
['83', '03/23', '64', '19', '82', '03/06']
low data
['26', '03/20', '43', '-17', '26', '03/12']


high data
['89', '04/04', '73', '16', '85', '04/24', '\n', '04/29']
low data
['41', '04/24', '50', '-9', '33', '04/01']


high data
['87', '05/08', '76', '11', '91', '05/12']
low data
['43', '05/01', '56', '-13', '45', '05/04']


high data
['100', '06/29', '89', '11', '100', '06/22', '\n', '06/30', '\n', '06/25']
low data
['66', '06/13', '68', '-2', '61', '06/03']


high data
['102', '07/29', '93', '9', '103', '07/26']
low data
['67', '07/10', '72', '-5', '71', '07/15', '\n', '07/11']


high data
['107', '08/26', '92', '15', '104', '08/15']
low data
['65', '08/17', '71', '-6', '67', '08/19', '\n', '08/14']


hi

{'mon': ['JAN',
  'FEB',
  'MAR',
  'APR',
  'MAY',
  'JUN',
  'JUL',
  'AUG',
  'SEP',
  'OCT',
  'NOV',
  'DEC'],
 'highs': ['77',
  '81',
  '83',
  '89',
  '87',
  '100',
  '102',
  '107',
  '102',
  '95',
  '83',
  '69'],
 'high_dates': [['01/11'],
  ['02/28'],
  ['03/23'],
  ['04/04'],
  ['05/08'],
  ['06/29'],
  ['07/29'],
  ['08/26'],
  ['09/06'],
  ['10/01'],
  ['11/06'],
  ['12/01']],
 'lows': ['25',
  '27',
  '26',
  '41',
  '43',
  '66',
  '67',
  '65',
  '59',
  '34',
  '29',
  '29'],
 'low_dates': [['01/14'],
  ['02/18'],
  ['03/20'],
  ['04/24'],
  ['05/01'],
  ['06/13'],
  ['07/10'],
  ['08/17'],
  ['09/18'],
  ['10/31'],
  ['11/27'],
  ['12/19']]}

In [150]:
# establish our main working dictionary
data_dict = {}

for i in range(1,13):
    _data_dict = {
         f'{calendar.month_name[i][:3].upper()}' : {# generate the abbreviated month names
        'highs': [],
        'high_dates': [], # these should still contain 12 items, but will be lists inside lists in case of multiple dates
        'lows': [],
        'low_dates': []  # these should still contain 12 items, but will be lists inside lists in case of multiple dates
        }
    }
    data_dict.update(_data_dict)

data_dict

{'JAN': {'highs': [], 'high_dates': [], 'lows': [], 'low_dates': []},
 'FEB': {'highs': [], 'high_dates': [], 'lows': [], 'low_dates': []},
 'MAR': {'highs': [], 'high_dates': [], 'lows': [], 'low_dates': []},
 'APR': {'highs': [], 'high_dates': [], 'lows': [], 'low_dates': []},
 'MAY': {'highs': [], 'high_dates': [], 'lows': [], 'low_dates': []},
 'JUN': {'highs': [], 'high_dates': [], 'lows': [], 'low_dates': []},
 'JUL': {'highs': [], 'high_dates': [], 'lows': [], 'low_dates': []},
 'AUG': {'highs': [], 'high_dates': [], 'lows': [], 'low_dates': []},
 'SEP': {'highs': [], 'high_dates': [], 'lows': [], 'low_dates': []},
 'OCT': {'highs': [], 'high_dates': [], 'lows': [], 'low_dates': []},
 'NOV': {'highs': [], 'high_dates': [], 'lows': [], 'low_dates': []},
 'DEC': {'highs': [], 'high_dates': [], 'lows': [], 'low_dates': []}}

In [112]:
print(text)
    

    

447 
CXUS54 KLZK 151933
CLMLIT

CLIMATE REPORT...UPDATED WITH SUPPLEMENTAL DATA 
NATIONAL WEATHER SERVICE LITTLE ROCK AR
0135 PM CST MON JAN 15 2024

...................................

...THE LITTLE ROCK ADAMS FIELD CLIMATE SUMMARY FOR THE MONTH OF DECEMBER 2023...

CLIMATE NORMAL PERIOD 1991 TO 2020
CLIMATE RECORD PERIOD 1874 TO 2023

WEATHER         OBSERVED          NORMAL  DEPART   LAST YEAR'S         
                VALUE   DATE(S)   VALUE   FROM     VALUE DATE(S)       
                                          NORMAL                       
................................................................
TEMPERATURE (F)
RECORD
 HIGH             81   12/26/2015                                      
 LOW              -1   12/23/1989                                      
                       12/24/1963                                      
HIGHEST           69   12/01         53      16       74  12/29        
LOWEST            29   12/19         34      -5        2  12/23     

In [89]:
test = high

test = [x for x in test if not test == '']
test

['',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '69',
 '',
 '',
 '12/01',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '53',
 '',
 '',
 '',
 '',
 '',
 '16',
 '',
 '',
 '',
 '',
 '',
 '',
 '74',
 '',
 '12/29',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '\nLOWEST',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '29',
 '',
 '',
 '12/19',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '34',
 '',
 '',
 '',
 '',
 '',
 '-5',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '2',
 '',
 '12/23',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '\nAVG.',
 'MAXIMUM',
 '',
 '',
 '',
 '59.8',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '52.6',
 '',
 '',
 '',
 '',
 '7.2',
 '',
 '',
 '',
 '',
 '53.9',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '\nAVG.',
 'MINIMUM',
 '',
 '',
 '',
 '37.9',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '33.3',
 '',
 '',
 '',
 '',
 '4.6',
 '',
 '',
 '',
 '',
 '36.5',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 

In [90]:
test = list(filter(None, test))
test

['69',
 '12/01',
 '53',
 '16',
 '74',
 '12/29',
 '\nLOWEST',
 '29',
 '12/19',
 '34',
 '-5',
 '2',
 '12/23',
 '\nAVG.',
 'MAXIMUM',
 '59.8',
 '52.6',
 '7.2',
 '53.9',
 '\nAVG.',
 'MINIMUM',
 '37.9',
 '33.3',
 '4.6',
 '36.5',
 '\nMEAN',
 '48.8',
 '43.0',
 '5.8',
 '45.2',
 '\nDAYS',
 'MAX',
 '>=',
 '90',
 '0',
 '0.0',
 '0.0',
 '0',
 '\nDAYS',
 'MAX',
 '<=',
 '32',
 '0',
 '0.9',
 '-0.9',
 '1',
 '\nDAYS',
 'MIN',
 '<=',
 '32',
 '7',
 '13.4',
 '-6.4',
 '12',
 '\nDAYS',
 'MIN',
 '<=',
 '0',
 '0',
 '0.0',
 '0.0',
 '0',
 '\n']

In [87]:
while ("" in test):
   test.remove("")

test

['6912/0153167412/29',
 'LOWEST2912/1934-5212/23',
 'AVG.MAXIMUM59.852.67.253.9',
 'AVG.MINIMUM37.933.34.636.5',
 'MEAN48.843.05.845.2',
 'DAYSMAX>=9000.00.00',
 'DAYSMAX<=3200.9-0.91',
 'DAYSMIN<=32713.4-6.412',
 'DAYSMIN<=000.00.00']

In [ ]:
for item in high:

In [86]:
high

['',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '69',
 '',
 '',
 '12/01',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '53',
 '',
 '',
 '',
 '',
 '',
 '16',
 '',
 '',
 '',
 '',
 '',
 '',
 '74',
 '',
 '12/29',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '\nLOWEST',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '29',
 '',
 '',
 '12/19',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '34',
 '',
 '',
 '',
 '',
 '',
 '-5',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '2',
 '',
 '12/23',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '\nAVG.',
 'MAXIMUM',
 '',
 '',
 '',
 '59.8',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '52.6',
 '',
 '',
 '',
 '',
 '7.2',
 '',
 '',
 '',
 '',
 '53.9',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '\nAVG.',
 'MINIMUM',
 '',
 '',
 '',
 '37.9',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '33.3',
 '',
 '',
 '',
 '',
 '4.6',
 '',
 '',
 '',
 '',
 '36.5',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 

In [64]:
import time

In [68]:
data_dict

{'mon': ['jan',
  'feb',
  'mar',
  'apr',
  'may',
  'jun',
  'jul',
  'aug',
  'sep',
  'oct',
  'nov',
  'dec'],
 'highs': ['69']}

In [73]:
m=[calendar.month_name[i][:3].upper() for i in range(1,13)]
m

['JAN',
 'FEB',
 'MAR',
 'APR',
 'MAY',
 'JUN',
 'JUL',
 'AUG',
 'SEP',
 'OCT',
 'NOV',
 'DEC']

In [70]:
import calendar